In [69]:
# Import library
import re
import sys
import os
import shutil
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr
import time
import pandas as pd

In [70]:
def relatedness(model):
    # Baca file rw untuk evaluasi word embedding dengan teknik relatedness cbow
    dir_file = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataEvaluasiGitHub/word-similarity/monolingual/en/rw.csv'
    list_model_similarity = []
    words_exist = []
    words_not_exist = []
    list_human_score = []
    
    df = pd.read_csv(dir_file)
    list_human_score = df['similarity'].to_list()
    
    for index, row in df.iterrows():
        word_1 = row["word1"].lower()
        word_2 = row["word2"].lower()
        try:
            wv_1 = []
            wv_2 = []
            wv_1.append(model.get_word_vector(word_1))
            
            if word_1 not in words_exist:
                words_exist.append(word_1)
            
            wv_2.append(model.get_word_vector(word_2))
            
            if word_2 not in words_exist:
                words_exist.append(word_2)
                
            similarity = cosine_similarity(wv_1, wv_2)[0][0]
            list_model_similarity.append(similarity)
            
        except:
            if word_1 in words_exist: # kata 2 yang tidak ada di model
                if word_2 not in words_not_exist:
                    words_not_exist.append(word_2)
            
            if word_2 in words_exist: # kata 1 yang tidak ada di model
                if word_1 not in words_not_exist:
                    words_not_exist.append(word_1)
        
    
   
        

    rho_model, p_model = spearmanr(list_model_similarity, list_human_score)
    print("banyak kata ada di model", len(words_exist))
    print("banyak kata tidak ada di model", len(words_not_exist))
    return rho_model


In [71]:
def word_analogy(model):
    # Baca file word_analogy untuk evaluasi word embedding dengan teknik word analogy (menggunakan library)

    dir_file = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataEvaluasiGitHub/word-analogy/monolingual/en/semeval.csv'
    df = pd.read_csv(dir_file)
    true_predicted = 0
    false_predicted = 0
    words_exist = []
    words_not_exist = []
    
    for index, row in df.iterrows():
        word_1 = row["word1"].lower()
        word_2 = row["word2"].lower()
        word_3 = row["word3"].lower()
        word_4 = row["target"].lower()
        word_predicted_curr = ""
        try:
            list_of_result_analogy = model.get_analogies(word_1, word_2, word_3)
            word_predicted_curr = list_of_result_analogy[0][1] 
            if word_predicted_curr == word_4:
                true_predicted += 1
            else:
                false_predicted += 1
            
            if word_1 not in words_exist:
                words_exist.append(word_1)
            
            if word_2 not in words_exist:
                words_exist.append(word_2)
            
            if word_3 not in words_exist:
                words_exist.append(word_3)
            
            if word_4 not in words_exist:
                words_exist.append(word_4)

        except:
            continue
            
        # untuk menambahkan word_1 jika tidak ada di model    
        try:
            word_vector = model.get_word_vector(word_1)
        except:
            if word_1 not in words_not_exist:
                    words_not_exist.append(word_1)
        
        # untuk menambahkan word_2 jika tidak ada di model    
        try:
            word_vector = model.get_word_vector(word_2)
        except:
            if word_2 not in words_not_exist:
                    words_not_exist.append(word_2)

        # untuk menambahkan word_3 jika tidak ada di model    
        try:
            word_vector = model.get_word_vector(word_3)
        except:
            if word_3 not in words_not_exist:
                    words_not_exist.append(word_3)
        
        # untuk menambahkan word_4 jika tidak ada di model    
        try:
            word_vector = model.get_word_vector(word_4)
        except:
            if word_4 not in words_not_exist:
                    words_not_exist.append(word_4)

    print("true_predicted", true_predicted, true_predicted / (true_predicted+false_predicted))
    print("false_predicted", false_predicted, false_predicted / (true_predicted+false_predicted))
    print("banyak kata ada di model", len(words_exist))
    print("banyak kata tidak ada di model", len(words_not_exist))

In [72]:
# Ambil jumlah paragraf dari file asli per dokumen
directory_path_real = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap7(DataCleaning)'

files_real = os.listdir(directory_path_real)
list_files_real = []
all_text_real = ""

# Ambil daftar nama file
for file in files_real:
    if '.DS_Store' not in file:
        list_files_real.append(file)
        
all_text_real = ""
for file in list_files_real:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/DataPreprocessing/DataTahap7(DataCleaning)/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_real += content


In [73]:
# Ambil jumlah paragraf dari file parafrasa per dokumen
directory_path_paraphrased = '/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_parafrasa'

files_paraphrased = os.listdir(directory_path_paraphrased)
list_files_paraphrased = []
all_text_paraphrased = ""

# Ambil daftar nama file
for file in files_paraphrased:
    if '.DS_Store' not in file:
        list_files_paraphrased.append(file)
        
all_text_paraphrased = ""
for file in list_files_paraphrased:
    with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/Data_parafrasa/' + file, 'r') as fileNow:
        content = fileNow.read()
        all_text_paraphrased += content
        


In [74]:
# Melakukan lowercase lagi
all_text_real = all_text_real.lower()
all_text_paraphrased = all_text_paraphrased.lower()

In [75]:
# Menggabungkan teks asli dan parafrasa
all_text = all_text_real + '\n' + all_text_paraphrased

In [76]:
# Hanya mengambil huruf dan titik
all_text = re.sub(r'[^a-zA-Z.]', ' ', all_text)
all_text = re.sub(r'\s+', ' ', all_text)

In [77]:
# Buat file sementara
with open('/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', 'w') as file:
    file.write(all_text)

In [49]:
# Membuat model word embedding fasttext menggunakan CBOW 1

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.01, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress:  99.9% words/sec/thread:  196221 lr:  0.000005 avg.loss:  1.779276 ETA:   0h 0m 0s  4.4% words/sec/thread:  289668 lr:  0.009561 avg.loss:  2.306376 ETA:   0h 2m22s  5.3% words/sec/thread:  284993 lr:  0.009466 avg.loss:  2.265436 ETA:   0h 2m22s0.009145 avg.loss:  2.182118 ETA:   0h 2m21s 12.6% words/sec/thread:  262765 lr:  0.008740 avg.loss:  2.094773 ETA:   0h 2m23savg.loss:  2.080030 ETA:   0h 2m25s 14.9% words/sec/thread:  247337 lr:  0.008510 avg.loss:  2.063075 ETA:   0h 2m28s 20.6% words/sec/thread:  238072 lr:  0.007943 avg.loss:  2.020056 ETA:   0h 2m23s 24.4% words/sec/thread:  230981 lr:  0.007556 avg.loss:  1.997369 ETA:   0h 2m20s 26.4% words/sec/thread:  226428 lr:  0.007358 avg.loss:  1.989278 ETA:   0h 2m19s 27.0% words/sec/thread:  226072 lr:  0.007304 avg.loss:  1.987154 ETA:   0h 2m19s 29.2% words/sec/thread:  224063 lr:  0.007084 avg.loss:  1.981219 ETA:   0h 2m16s% words/sec/thread:  220074 lr:  

Total training time: 0 hours 3 minutes 40 seconds


Progress: 100.0% words/sec/thread:  196238 lr:  0.000000 avg.loss:  1.779026 ETA:   0h 0m 0s


In [50]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_1.bin")


In [51]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_1.bin")


In [52]:
# relatedness model cbow 1
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.18404157070874463


In [53]:
# word analogy model cbow 1
word_analogy(model_cbow)

true_predicted 1 9.986019572598362e-05
false_predicted 10013 0.9999001398042741
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [54]:
# Membuat model word embedding fasttext menggunakan skip gram 1

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.01, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress: 100.0% words/sec/thread:   70146 lr:  0.000000 avg.loss:  0.700793 ETA:   0h 0m 0s  2.0% words/sec/thread:  120453 lr:  0.009801 avg.loss:  2.090265 ETA:   0h 5m50s  3.4% words/sec/thread:  111408 lr:  0.009662 avg.loss:  2.075347 ETA:   0h 6m13s  4.4% words/sec/thread:  106813 lr:  0.009555 avg.loss:  2.069773 ETA:   0h 6m24s  6.0% words/sec/thread:   98863 lr:  0.009401 avg.loss:  2.068552 ETA:   0h 6m49s  6.1% words/sec/thread:   98532 lr:  0.009389 avg.loss:  2.069732 ETA:   0h 6m49s  6.7% words/sec/thread:   95575 lr:  0.009332 avg.loss:  2.074458 ETA:   0h 7m 0savg.loss:  2.076156 ETA:   0h 7m 9s  7.8% words/sec/thread:   89922 lr:  0.009220 avg.loss:  2.081098 ETA:   0h 7m21s  8.2% words/sec/thread:   88669 lr:  0.009179 avg.loss:  2.083637 ETA:   0h 7m25s  8.2% words/sec/thread:   88641 lr:  0.009177 avg.loss:  2.083841 ETA:   0h 7m25s  8.5% words/sec/thread:   88462 lr:  0.009147 avg.loss:  2.084312 ETA:   0h 

Total training time: 0 hours 10 minutes 14 seconds


In [55]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_1.bin")


In [56]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_1.bin")


In [57]:
# relatedness model sg 1
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.19614537472060697


In [58]:
# word analogy model sg 1
word_analogy(model_sg)

true_predicted 9 0.0008987417615338526
false_predicted 10005 0.9991012582384662
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [59]:
# Membuat model word embedding fasttext menggunakan CBOW 2

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.001, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress: 100.0% words/sec/thread:  137938 lr:  0.000000 avg.loss:  2.090869 ETA:   0h 0m 0s  2.4% words/sec/thread:  267271 lr:  0.000976 avg.loss:  3.403772 ETA:   0h 2m37s  7.6% words/sec/thread:  244464 lr:  0.000924 avg.loss:  2.700732 ETA:   0h 2m42s 11.2% words/sec/thread:  228339 lr:  0.000888 avg.loss:  2.586280 ETA:   0h 2m47s 18.1% words/sec/thread:  214613 lr:  0.000819 avg.loss:  2.479659 ETA:   0h 2m44s 20.4% words/sec/thread:  209762 lr:  0.000796 avg.loss:  2.453553 ETA:   0h 2m43s 21.9% words/sec/thread:  206488 lr:  0.000781 avg.loss:  2.438140 ETA:   0h 2m42s 23.3% words/sec/thread:  204844 lr:  0.000767 avg.loss:  2.424301 ETA:   0h 2m41s 25.3% words/sec/thread:  203120 lr:  0.000747 avg.loss:  2.404988 ETA:   0h 2m38s 26.6% words/sec/thread:  201346 lr:  0.000734 avg.loss:  2.393816 ETA:   0h 2m36s 31.5% words/sec/thread:  198568 lr:  0.000685 avg.loss:  2.362891 ETA:   0h 2m28s 32.7% words/sec/thread:  1942

Total training time: 0 hours 5 minutes 12 seconds


Progress: 100.0% words/sec/thread:  137906 lr:  0.000000 avg.loss:  2.090842 ETA:   0h 0m 0s


In [60]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_2.bin")


In [61]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_2.bin")


In [62]:
# relatedness model cbow 2
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.16557757338755275


In [63]:
# word analogy model cbow 2
word_analogy(model_cbow)

true_predicted 0 0.0
false_predicted 10014 1.0
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [64]:
# Membuat model word embedding fasttext menggunakan skip gram 2

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.001, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress: 100.0% words/sec/thread:   69288 lr:  0.000000 avg.loss:  0.743549 ETA:   0h 0m 0s  1.8% words/sec/thread:  129548 lr:  0.000982 avg.loss:  2.941371 ETA:   0h 5m25savg.loss:  2.798698 ETA:   0h 5m40s  4.2% words/sec/thread:  108471 lr:  0.000958 avg.loss:  2.571348 ETA:   0h 6m19s  5.0% words/sec/thread:  108604 lr:  0.000950 avg.loss:  2.528103 ETA:   0h 6m16s  5.7% words/sec/thread:  107371 lr:  0.000943 avg.loss:  2.501015 ETA:   0h 6m17s  6.2% words/sec/thread:  104855 lr:  0.000938 avg.loss:  2.486565 ETA:   0h 6m24savg.loss:  2.476607 ETA:   0h 6m31s 7.0% words/sec/thread:  101213 lr:  0.000930 avg.loss:  2.465971 ETA:   0h 6m35s  8.6% words/sec/thread:   97813 lr:  0.000914 avg.loss:  2.436626 ETA:   0h 6m41s  0.000907 avg.loss:  2.426136 ETA:   0h 6m52s  9.9% words/sec/thread:   93647 lr:  0.000901 avg.loss:  2.416547 ETA:   0h 6m54s 11.0% words/sec/thread:   90153 lr:  0.000890 avg.loss:  2.392697 ETA:   0h 7m

Total training time: 0 hours 10 minutes 22 seconds


Progress: 100.0% words/sec/thread:   69278 lr:  0.000000 avg.loss:  0.743535 ETA:   0h 0m 0s


In [65]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_2.bin")


In [66]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_2.bin")


In [67]:
# relatedness model sg 2
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.21906551118069492


In [68]:
# word analogy model sg 2
word_analogy(model_sg)

true_predicted 5 0.0004993009786299181
false_predicted 10009 0.9995006990213701
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [78]:
# Membuat model word embedding fasttext menggunakan CBOW 3

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.1, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress:  99.9% words/sec/thread:  205886 lr:  0.000103 avg.loss:  1.673700 ETA:   0h 0m 0s  4.0% words/sec/thread:  310853 lr:  0.096010 avg.loss:  2.097997 ETA:   0h 2m12s  5.0% words/sec/thread:  297037 lr:  0.094997 avg.loss:  2.076303 ETA:   0h 2m17s  6.4% words/sec/thread:  289698 lr:  0.093601 avg.loss:  2.058693 ETA:   0h 2m19s words/sec/thread:  287787 lr:  0.092550 avg.loss:  2.046057 ETA:   0h 2m18s 0.086194 avg.loss:  1.990080 ETA:   0h 2m13srogress:  15.1% words/sec/thread:  275502 lr:  0.084907 avg.loss:  1.984575 ETA:   0h 2m12s 16.8% words/sec/thread:  275045 lr:  0.083165 avg.loss:  1.977442 ETA:   0h 2m10s% words/sec/thread:  273548 lr:  0.080804 avg.loss:  1.965926 ETA:   0h 2m 7s 23.9% words/sec/thread:  272993 lr:  0.076116 avg.loss:  1.945600 ETA:   0h 1m59s 33.0% words/sec/thread:  263425 lr:  0.067006 avg.loss:  1.917091 ETA:   0h 1m49s 34.3% words/sec/thread:  262138 lr:  0.065739 avg.loss:  1.913362 ET

Total training time: 0 hours 3 minutes 29 seconds


Progress: 100.0% words/sec/thread:  205902 lr:  0.000000 avg.loss:  1.673203 ETA:   0h 0m 0s


In [79]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_3.bin")


In [80]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_3.bin")


In [81]:
# relatedness model cbow 3
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.21762753307429641


In [82]:
# word analogy model cbow 3
word_analogy(model_cbow)

true_predicted 17 0.0016976233273417215
false_predicted 9997 0.9983023766726583
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [83]:
# Membuat model word embedding fasttext menggunakan skip gram 3

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.1, dim=20, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  84.4% words/sec/thread:   63094 lr:  0.015618 avg.loss:  0.768366 ETA:   0h 1m46s

Progress: 100.0% words/sec/thread:   61075 lr:  0.000006 avg.loss:  0.687828 ETA:   0h 0m 0s 92.9% words/sec/thread:   62116 lr:  0.007057 avg.loss:  0.720893 ETA:   0h 0m48s 93.9% words/sec/thread:   61948 lr:  0.006051 avg.loss:  0.715812 ETA:   0h 0m42s 94.6% words/sec/thread:   61805 lr:  0.005361 avg.loss:  0.712418 ETA:   0h 0m37s 97.9% words/sec/thread:   61357 lr:  0.002094 avg.loss:  0.696949 ETA:   0h 0m14s 99.5% words/sec/thread:   61224 lr:  0.000486 avg.loss:  0.689859 ETA:   0h 0m 3s

Total training time: 0 hours 11 minutes 45 seconds


Progress: 100.0% words/sec/thread:   61071 lr:  0.000000 avg.loss:  0.687777 ETA:   0h 0m 0s


In [84]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_3.bin")


In [85]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_3.bin")


In [86]:
# relatedness model sg 3
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.17792551181090308


In [87]:
# word analogy model sg 3
word_analogy(model_sg)

true_predicted 9 0.0008987417615338526
false_predicted 10005 0.9991012582384662
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [88]:
# Membuat model word embedding fasttext menggunakan CBOW 4

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.01, dim=10, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress:  99.9% words/sec/thread:  169177 lr:  0.000012 avg.loss:  1.894973 ETA:   0h 0m 0s  5.7% words/sec/thread:  407134 lr:  0.009431 avg.loss:  2.264205 ETA:   0h 1m39s 10.4% words/sec/thread:  370415 lr:  0.008965 avg.loss:  2.173385 ETA:   0h 1m44s 13.7% words/sec/thread:  352232 lr:  0.008631 avg.loss:  2.137774 ETA:   0h 1m45s 32.4% words/sec/thread:  336340 lr:  0.006758 avg.loss:  2.042386 ETA:   0h 1m26s 34.2% words/sec/thread:  321803 lr:  0.006578 avg.loss:  2.036424 ETA:   0h 1m27s 37.0% words/sec/thread:  302395 lr:  0.006295 avg.loss:  2.032134 ETA:   0h 1m29s 41.3% words/sec/thread:  274345 lr:  0.005871 avg.loss:  2.022236 ETA:   0h 1m32s 41.8% words/sec/thread:  270487 lr:  0.005819 avg.loss:  2.022203 ETA:   0h 1m32ss 45.8% words/sec/thread:  247005 lr:  0.005421 avg.loss:  2.022238 ETA:   0h 1m34s 46.6% words/sec/thread:  243037 lr:  0.005338 avg.loss:  2.022503 ETA:   0h 1m34s 47.1% words/sec/thread:  240

Total training time: 0 hours 4 minutes 15 seconds


Progress: 100.0% words/sec/thread:  169116 lr:  0.000000 avg.loss:  1.894528 ETA:   0h 0m 0s


In [89]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_4.bin")


In [90]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_4.bin")


In [91]:
# relatedness model cbow 4
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.14085349454666213


In [92]:
# word analogy model cbow 4
word_analogy(model_cbow)

true_predicted 1 9.986019572598362e-05
false_predicted 10013 0.9999001398042741
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [93]:
# Membuat model word embedding fasttext menggunakan skip gram 4

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.01, dim=10, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  77.6% words/sec/thread:   55525 lr:  0.002241 avg.loss:  0.814277 ETA:   0h 2m53s

Progress: 100.0% words/sec/thread:   55507 lr:  0.000001 avg.loss:  0.694403 ETA:   0h 0m 0s 77.7% words/sec/thread:   55521 lr:  0.002231 avg.loss:  0.813600 ETA:   0h 2m52s 77.9% words/sec/thread:   55517 lr:  0.002210 avg.loss:  0.812260 ETA:   0h 2m51s 78.3% words/sec/thread:   55491 lr:  0.002173 avg.loss:  0.809870 ETA:   0h 2m48s 78.7% words/sec/thread:   55468 lr:  0.002132 avg.loss:  0.807222 ETA:   0h 2m45s 79.1% words/sec/thread:   55461 lr:  0.002092 avg.loss:  0.804390 ETA:   0h 2m42s 79.5% words/sec/thread:   55426 lr:  0.002054 avg.loss:  0.801641 ETA:   0h 2m39s 79.7% words/sec/thread:   55415 lr:  0.002034 avg.loss:  0.800271 ETA:   0h 2m37s 0.002017 avg.loss:  0.799021 ETA:   0h 2m36s 80.0% words/sec/thread:   55364 lr:  0.001996 avg.loss:  0.797585 ETA:   0h 2m35s 80.4% words/sec/thread:   55323 lr:  0.001956 avg.loss:  0.795112 ETA:   0h 2m32s 80.9% words/sec/thread:   55308 lr:  0.001915 avg.loss:  0.792505 ETA:   0h 2m28s 81.3% words/sec/thread:   55276 lr:  0.001

Total training time: 0 hours 12 minutes 57 seconds


Progress: 100.0% words/sec/thread:   55507 lr:  0.000000 avg.loss:  0.694333 ETA:   0h 0m 0s


In [94]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_4.bin")


In [95]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_4.bin")


In [96]:
# relatedness model sg 4
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.14371061662628598


In [97]:
# word analogy model sg 4
word_analogy(model_sg)

true_predicted 0 0.0
false_predicted 10014 1.0
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [98]:
# Membuat model word embedding fasttext menggunakan CBOW 5

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.001, dim=10, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress: 100.0% words/sec/thread:  189744 lr:  0.000000 avg.loss:  2.145473 ETA:   0h 0m 0ss 13.3% words/sec/thread:  157690 lr:  0.000867 avg.loss:  2.554197 ETA:   0h 3m56s 15.2% words/sec/thread:  156675 lr:  0.000848 avg.loss:  2.523060 ETA:   0h 3m52s 17.9% words/sec/thread:  155448 lr:  0.000821 avg.loss:  2.489578 ETA:   0h 3m47s 19.0% words/sec/thread:  154774 lr:  0.000810 avg.loss:  2.478331 ETA:   0h 3m45s 19.5% words/sec/thread:  154188 lr:  0.000805 avg.loss:  2.472007 ETA:   0h 3m44s 20.5% words/sec/thread:  153606 lr:  0.000795 avg.loss:  2.460956 ETA:   0h 3m42s 21.6% words/sec/thread:  153472 lr:  0.000784 avg.loss:  2.450231 ETA:   0h 3m39s 22.1% words/sec/thread:  153350 lr:  0.000779 avg.loss:  2.445560 ETA:   0h 3m38s ETA:   0h 3m37s 26.5% words/sec/thread:  153263 lr:  0.000735 avg.loss:  2.409596 ETA:   0h 3m26s 0.000731 avg.loss:  2.407049 ETA:   0h 3m25s 27.0% words/sec/thread:  153481 lr:  0.000730 avg

Total training time: 0 hours 3 minutes 49 seconds


Progress: 100.0% words/sec/thread:  189660 lr:  0.000000 avg.loss:  2.145336 ETA:   0h 0m 0s


In [99]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_5.bin")


In [100]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_5.bin")


In [101]:
# relatedness model cbow 5
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.1418965273649628


In [102]:
# word analogy model cbow 5
word_analogy(model_cbow)

true_predicted 0 0.0
false_predicted 10014 1.0
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [103]:
# Membuat model word embedding fasttext menggunakan skip gram 5

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.001, dim=10, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress: 100.0% words/sec/thread:   61331 lr:  0.000000 avg.loss:  0.740764 ETA:   0h 0m 0s  0.2% words/sec/thread:   68544 lr:  0.000998 avg.loss:  4.131942 ETA:   0h10m26s  0h10m54s  3.9% words/sec/thread:   61960 lr:  0.000961 avg.loss:  2.607830 ETA:   0h11m 7s  7.4% words/sec/thread:   61806 lr:  0.000926 avg.loss:  2.463120 ETA:   0h10m44s  8.3% words/sec/thread:   62050 lr:  0.000917 avg.loss:  2.444732 ETA:   0h10m35s  8.7% words/sec/thread:   61807 lr:  0.000913 avg.loss:  2.438807 ETA:   0h10m35s  9.2% words/sec/thread:   61231 lr:  0.000908 avg.loss:  2.430479 ETA:   0h10m38s  9.7% words/sec/thread:   61218 lr:  0.000903 avg.loss:  2.421318 ETA:   0h10m34s 10.1% words/sec/thread:   61248 lr:  0.000899 avg.loss:  2.413869 ETA:   0h10m31s 10.2% words/sec/thread:   61242 lr:  0.000898 avg.loss:  2.409988 ETA:   0h10m30s 10.6% words/sec/thread:   61367 lr:  0.000894 avg.loss:  2.402288 ETA:   0h10m27s avg.loss:  2.386923

Total training time: 0 hours 11 minutes 42 seconds


Progress: 100.0% words/sec/thread:   61335 lr:  0.000000 avg.loss:  0.740611 ETA:   0h 0m 0s


In [104]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_5.bin")


In [105]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_5.bin")


In [106]:
# relatedness model sg 5
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.14391645339097717


In [107]:
# word analogy model sg 5
word_analogy(model_sg)

true_predicted 0 0.0
false_predicted 10014 1.0
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [118]:
# Membuat model word embedding fasttext menggunakan CBOW 6

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.1, dim=10, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0
Progress:  99.9% words/sec/thread:  149774 lr:  0.000091 avg.loss:  1.828497 ETA:   0h 0m 0s  0.4% words/sec/thread:  127729 lr:  0.099615 avg.loss:  2.501026 ETA:   0h 5m35s 5m 4s  2.3% words/sec/thread:  142754 lr:  0.097740 avg.loss:  2.196236 ETA:   0h 4m54s  3.4% words/sec/thread:  141225 lr:  0.096634 avg.loss:  2.152895 ETA:   0h 4m54s  3.9% words/sec/thread:  141663 lr:  0.096089 avg.loss:  2.138064 ETA:   0h 4m51s  5.0% words/sec/thread:  143743 lr:  0.095009 avg.loss:  2.117526 ETA:   0h 4m44s  7.6% words/sec/thread:  148386 lr:  0.092423 avg.loss:  2.091191 ETA:   0h 4m27s  8.4% words/sec/thread:  148695 lr:  0.091595 avg.loss:  2.084688 ETA:   0h 4m25s 14.8% words/sec/thread:  154528 lr:  0.085213 avg.loss:  2.045392 ETA:   0h 3m57s 15.3% words/sec/thread:  154485 lr:  0.084670 avg.loss:  2.044062 ETA:   0h 3m55s2.041689 ETA:   0h 3m53s  0h 3m20s words/sec/thread:  158221 lr:  0.071906 avg.loss:  2.015633 ETA:   0h 3

Total training time: 0 hours 4 minutes 49 seconds


In [119]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_6.bin")


Progress: 100.0% words/sec/thread:  149752 lr:  0.000000 avg.loss:  1.828147 ETA:   0h 0m 0s


In [120]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_6.bin")


In [121]:
# relatedness model cbow 6
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.17569556117246327


In [122]:
# word analogy model cbow 6
word_analogy(model_cbow)

true_predicted 1 9.986019572598362e-05
false_predicted 10013 0.9999001398042741
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [123]:
# Membuat model word embedding fasttext menggunakan skip gram 6

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.1, dim=10, ws=5, epoch=100, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  68.7% words/sec/thread:   56779 lr:  0.031275 avg.loss:  0.917145 ETA:   0h 3m56s

Progress: 100.0% words/sec/thread:   57255 lr:  0.000004 avg.loss:  0.716649 ETA:   0h 0m 0s 68.8% words/sec/thread:   56767 lr:  0.031181 avg.loss:  0.916353 ETA:   0h 3m56s 69.1% words/sec/thread:   56760 lr:  0.030948 avg.loss:  0.914274 ETA:   0h 3m54s 70.3% words/sec/thread:   56758 lr:  0.029670 avg.loss:  0.903347 ETA:   0h 3m44s 76.4% words/sec/thread:   56897 lr:  0.023575 avg.loss:  0.856055 ETA:   0h 2m58s 82.3% words/sec/thread:   57190 lr:  0.017740 avg.loss:  0.814437 ETA:   0h 2m13s 82.4% words/sec/thread:   57173 lr:  0.017614 avg.loss:  0.813580 ETA:   0h 2m12s 83.5% words/sec/thread:   57201 lr:  0.016461 avg.loss:  0.805745 ETA:   0h 2m 3s 83.7% words/sec/thread:   57188 lr:  0.016275 avg.loss:  0.804499 ETA:   0h 2m 2s 90.6% words/sec/thread:   57357 lr:  0.009449 avg.loss:  0.765388 ETA:   0h 1m10s 91.5% words/sec/thread:   57340 lr:  0.008472 avg.loss:  0.759758 ETA:   0h 1m 3s words/sec/thread:   57342 lr:  0.008087 avg.loss:  0.757597 ETA:   0h 1m 0s 92.3% words

Total training time: 0 hours 12 minutes 33 seconds


Progress: 100.0% words/sec/thread:   57251 lr:  0.000000 avg.loss:  0.716628 ETA:   0h 0m 0s


In [124]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_6.bin")


In [125]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_6.bin")


In [126]:
# relatedness model sg 6
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.10906485675713878


In [127]:
# word analogy model sg 6
word_analogy(model_sg)

true_predicted 1 9.986019572598362e-05
false_predicted 10013 0.9999001398042741
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [128]:
# Membuat model word embedding fasttext menggunakan CBOW 7

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.1, dim=20, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  31.4% words/sec/thread:  133706 lr:  0.068645 avg.loss:  1.179023 ETA:   0h36m48s

Progress:  99.0% words/sec/thread:  139145 lr:  0.001026 avg.loss:  0.586406 ETA:   0h 0m31s 31.6% words/sec/thread:  133811 lr:  0.068385 avg.loss:  1.173273 ETA:   0h36m38s 31.9% words/sec/thread:  133791 lr:  0.068056 avg.loss:  1.165279 ETA:   0h36m28s 32.3% words/sec/thread:  133870 lr:  0.067694 avg.loss:  1.157069 ETA:   0h36m15s 32.3% words/sec/thread:  133788 lr:  0.067666 avg.loss:  1.156412 ETA:   0h36m16savg.loss:  1.155193 ETA:   0h36m15s 32.9% words/sec/thread:  133443 lr:  0.067144 avg.loss:  1.145364 ETA:   0h36m 4s% words/sec/thread:  133511 lr:  0.066617 avg.loss:  1.134640 ETA:   0h35m46s0.066478 avg.loss:  1.131914 ETA:   0h35m43s35m39s 33.8% words/sec/thread:  133099 lr:  0.066163 avg.loss:  1.125834 ETA:   0h35m38s avg.loss:  1.090575 ETA:   0h34m25s 38.9% words/sec/thread:  134501 lr:  0.061067 avg.loss:  1.045541 ETA:   0h32m33s 39.5% words/sec/thread:  134556 lr:  0.060491 avg.loss:  1.037744 ETA:   0h32m14s 40.6% words/sec/thread:  134975 lr:  0.059406 avg.los

Progress: 100.0% words/sec/thread:  139123 lr:  0.000002 avg.loss:  0.580522 ETA:   0h 0m 0s

Total training time: 0 hours 51 minutes 34 seconds


Progress: 100.0% words/sec/thread:  139121 lr:  0.000000 avg.loss:  0.580511 ETA:   0h 0m 0s


In [129]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_7.bin")


In [130]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_7.bin")


In [131]:
# relatedness model cbow 7
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.19966522703592046


In [132]:
# word analogy model cbow 7
word_analogy(model_cbow)

true_predicted 17 0.0016976233273417215
false_predicted 9997 0.9983023766726583
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [133]:
# Membuat model word embedding fasttext menggunakan skip gram 7

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.1, dim=20, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:   8.2% words/sec/thread:   48392 lr:  0.091794 avg.loss:  0.852166 ETA:   2h16m 1s

Progress:  24.3% words/sec/thread:   49356 lr:  0.075700 avg.loss:  0.405505 ETA:   1h49m58s ETA:   2h15m32s  8.4% words/sec/thread:   48472 lr:  0.091594 avg.loss:  0.843685 ETA:   2h15m30s  8.7% words/sec/thread:   48525 lr:  0.091335 avg.loss:  0.833770 ETA:   2h14m58s  8.9% words/sec/thread:   48579 lr:  0.091132 avg.loss:  0.826629 ETA:   2h14m31s  8.9% words/sec/thread:   48581 lr:  0.091092 avg.loss:  0.825065 ETA:   2h14m27s lr:  0.090895 avg.loss:  0.817844 ETA:   2h13m59s  9.5% words/sec/thread:   48795 lr:  0.090518 avg.loss:  0.805085 ETA:   2h13m 1s  9.5% words/sec/thread:   48808 lr:  0.090496 avg.loss:  0.804437 ETA:   2h12m57s  2h12m46s  9.7% words/sec/thread:   48823 lr:  0.090337 avg.loss:  0.799651 ETA:   2h12m40s  9.7% words/sec/thread:   48825 lr:  0.090303 avg.loss:  0.798666 ETA:   2h12m37s  9.9% words/sec/thread:   48891 lr:  0.090061 avg.loss:  0.791436 ETA:   2h12m 5s 10.2% words/sec/thread:   48956 lr:  0.089822 avg.loss:  0.784440 ETA:   2h11m33s 10.6% words

Progress:  33.1% words/sec/thread:   47652 lr:  0.066920 avg.loss:  0.301752 ETA:   1h40m42s 24.3% words/sec/thread:   49352 lr:  0.075690 avg.loss:  0.405350 ETA:   1h49m58s 24.4% words/sec/thread:   49312 lr:  0.075585 avg.loss:  0.403638 ETA:   1h49m54s 24.4% words/sec/thread:   49294 lr:  0.075551 avg.loss:  0.403098 ETA:   1h49m54s 24.5% words/sec/thread:   49270 lr:  0.075514 avg.loss:  0.402505 ETA:   1h49m54s 24.6% words/sec/thread:   49223 lr:  0.075425 avg.loss:  0.401071 ETA:   1h49m52s   49199 lr:  0.075399 avg.loss:  0.400678 ETA:   1h49m53s 24.7% words/sec/thread:   49089 lr:  0.075251 avg.loss:  0.398341 ETA:   1h49m55s 24.8% words/sec/thread:   49048 lr:  0.075164 avg.loss:  0.396995 ETA:   1h49m53s avg.loss:  0.396711 ETA:   1h49m52s 24.9% words/sec/thread:   49036 lr:  0.075131 avg.loss:  0.396489 ETA:   1h49m52s 24.9% words/sec/thread:   49013 lr:  0.075077 avg.loss:  0.395646 ETA:   1h49m50s 24.9% words/sec/thread:   49007 lr:  0.075061 avg.loss:  0.395403 ETA:   1h

Progress:  38.6% words/sec/thread:   47205 lr:  0.061390 avg.loss:  0.260689 ETA:   1h33m15s 33.2% words/sec/thread:   47660 lr:  0.066796 avg.loss:  0.300684 ETA:   1h40m29s 33.3% words/sec/thread:   47665 lr:  0.066705 avg.loss:  0.299919 ETA:   1h40m21s 33.5% words/sec/thread:   47679 lr:  0.066512 avg.loss:  0.298284 ETA:   1h40m 1s 33.9% words/sec/thread:   47724 lr:  0.066054 avg.loss:  0.294492 ETA:   1h39m15s 34.0% words/sec/thread:   47724 lr:  0.066007 avg.loss:  0.294102 ETA:   1h39m10savg.loss:  0.293956 ETA:   1h39m 9s 34.1% words/sec/thread:   47714 lr:  0.065920 avg.loss:  0.293399 ETA:   1h39m 4s 34.2% words/sec/thread:   47707 lr:  0.065835 avg.loss:  0.292704 ETA:   1h38m57s 34.2% words/sec/thread:   47704 lr:  0.065766 avg.loss:  0.292140 ETA:   1h38m51s48s 34.3% words/sec/thread:   47701 lr:  0.065714 avg.loss:  0.291718 ETA:   1h38m47s 34.3% words/sec/thread:   47698 lr:  0.065680 avg.loss:  0.291449 ETA:   1h38m44s90561 ETA:   1h38m36sc/thread:   47679 lr:  0.0655

...

Progress:  78.1% words/sec/thread:   44775 lr:  0.021902 avg.loss:  0.135774 ETA:   0h35m 4s 68.3% words/sec/thread:   42708 lr:  0.031716 avg.loss:  0.154019 ETA:   0h53m15savg.loss:  0.153983 ETA:   0h53m13s words/sec/thread:   42699 lr:  0.031645 avg.loss:  0.153866 ETA:   0h53m 8s 68.4% words/sec/thread:   42696 lr:  0.031613 avg.loss:  0.153800 ETA:   0h53m 5s 68.4% words/sec/thread:   42695 lr:  0.031595 avg.loss:  0.153761 ETA:   0h53m 3s 68.4% words/sec/thread:   42694 lr:  0.031577 avg.loss:  0.153724 ETA:   0h53m 2savg.loss:  0.153687 ETA:   0h53m 0s 68.5% words/sec/thread:   42685 lr:  0.031495 avg.loss:  0.153551 ETA:   0h52m54s 68.5% words/sec/thread:   42684 lr:  0.031476 avg.loss:  0.153513 ETA:   0h52m52s 68.6% words/sec/thread:   42682 lr:  0.031389 avg.loss:  0.153328 ETA:   0h52m44s 68.6% words/sec/thread:   42681 lr:  0.031371 avg.loss:  0.153291 ETA:   0h52m42s52m41s 68.7% words/sec/thread:   42676 lr:  0.031305 avg.loss:  0.153150 ETA:   0h52m36s 68.7% words/sec/t

Progress:  89.6% words/sec/thread:   47261 lr:  0.010369 avg.loss:  0.119510 ETA:   0h15m43s 78.1% words/sec/thread:   44776 lr:  0.021890 avg.loss:  0.135755 ETA:   0h35m 3s 78.2% words/sec/thread:   44787 lr:  0.021837 avg.loss:  0.135670 ETA:   0h34m57s 78.3% words/sec/thread:   44823 lr:  0.021694 avg.loss:  0.135444 ETA:   0h34m42savg.loss:  0.135381 ETA:   0h34m38s 78.4% words/sec/thread:   44851 lr:  0.021585 avg.loss:  0.135272 ETA:   0h34m30s 78.5% words/sec/thread:   44873 lr:  0.021490 avg.loss:  0.135121 ETA:   0h34m20savg.loss:  0.135059 ETA:   0h34m16s 78.7% words/sec/thread:   44917 lr:  0.021322 avg.loss:  0.134857 ETA:   0h34m 2s 0.021247 avg.loss:  0.134739 ETA:   0h33m54s words/sec/thread:   44938 lr:  0.021229 avg.loss:  0.134711 ETA:   0h33m52s 78.9% words/sec/thread:   44962 lr:  0.021121 avg.loss:  0.134538 ETA:   0h33m41s 78.9% words/sec/thread:   44972 lr:  0.021069 avg.loss:  0.134455 ETA:   0h33m35s 79.0% words/sec/thread:   45000 lr:  0.020964 avg.loss:  0.1

Progress: 100.0% words/sec/thread:   49157 lr:  0.000026 avg.loss:  0.107795 ETA:   0h 0m 2s 89.7% words/sec/thread:   47268 lr:  0.010337 avg.loss:  0.119471 ETA:   0h15m40s 89.9% words/sec/thread:   47310 lr:  0.010119 avg.loss:  0.119200 ETA:   0h15m20s 89.9% words/sec/thread:   47318 lr:  0.010081 avg.loss:  0.119152 ETA:   0h15m16s 90.1% words/sec/thread:   47347 lr:  0.009948 avg.loss:  0.118986 ETA:   0h15m 3s 0.118912 ETA:   0h14m58s 90.2% words/sec/thread:   47376 lr:  0.009815 avg.loss:  0.118822 ETA:   0h14m51s90.2% words/sec/thread:   47382 lr:  0.009779 avg.loss:  0.118776 ETA:   0h14m47s 90.4% words/sec/thread:   47413 lr:  0.009601 avg.loss:  0.118556 ETA:   0h14m31s 90.4% words/sec/thread:   47419 lr:  0.009554 avg.loss:  0.118499 ETA:   0h14m26s  0.009377 avg.loss:  0.118281 ETA:   0h14m10s 90.7% words/sec/thread:   47463 lr:  0.009281 avg.loss:  0.118163 ETA:   0h14m 1s 90.7% words/sec/thread:   47467 lr:  0.009253 avg.loss:  0.118128 ETA:   0h13m58s 90.9% words/sec/t

Total training time: 2 hours 25 minutes 54 seconds


Progress: 100.0% words/sec/thread:   49162 lr:  0.000000 avg.loss:  0.107769 ETA:   0h 0m 0s


In [134]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_7.bin")


In [135]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_7.bin")


In [136]:
# relatedness model sg 7
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.16238052201787823


In [137]:
# word analogy model sg 7
word_analogy(model_sg)

true_predicted 12 0.0011983223487118035
false_predicted 10002 0.9988016776512882
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [138]:
# Membuat model word embedding fasttext menggunakan CBOW 8

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.1, dim=30, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  19.3% words/sec/thread:  160796 lr:  0.080741 avg.loss:  1.565469 ETA:   0h36m 0s

Progress:  43.4% words/sec/thread:  161117 lr:  0.056597 avg.loss:  1.044999 ETA:   0h25m11s 19.3% words/sec/thread:  160788 lr:  0.080707 avg.loss:  1.563779 ETA:   0h35m59s 1.548617 ETA:   0h35m52s 19.7% words/sec/thread:  160700 lr:  0.080328 avg.loss:  1.545565 ETA:   0h35m50s 19.9% words/sec/thread:  160514 lr:  0.080128 avg.loss:  1.535930 ETA:   0h35m47s 20.1% words/sec/thread:  160618 lr:  0.079872 avg.loss:  1.524085 ETA:   0h35m39savg.loss:  1.520504 ETA:   0h35m37s20.3% words/sec/thread:  160593 lr:  0.079709 avg.loss:  1.516666 ETA:   0h35m35s 20.4% words/sec/thread:  160619 lr:  0.079613 avg.loss:  1.512350 ETA:   0h35m32s 20.6% words/sec/thread:  160573 lr:  0.079396 avg.loss:  1.502734 ETA:   0h35m27s 20.7% words/sec/thread:  160665 lr:  0.079280 avg.loss:  1.497391 ETA:   0h35m23s 20.9% words/sec/thread:  160788 lr:  0.079110 avg.loss:  1.489813 ETA:   0h35m16s 21.2% words/sec/thread:  160779 lr:  0.078844 avg.loss:  1.478313 ETA:   0h35m 9s 21.3% words/sec/thread:  160

Progress:  67.5% words/sec/thread:  158693 lr:  0.032523 avg.loss:  0.837101 ETA:   0h14m41s 43.4% words/sec/thread:  161129 lr:  0.056551 avg.loss:  1.044515 ETA:   0h25m10s 43.5% words/sec/thread:  161149 lr:  0.056468 avg.loss:  1.043617 ETA:   0h25m 7s 43.6% words/sec/thread:  161164 lr:  0.056402 avg.loss:  1.043039 ETA:   0h25m 5s 44.1% words/sec/thread:  161265 lr:  0.055897 avg.loss:  1.038729 ETA:   0h24m51s 44.5% words/sec/thread:  161381 lr:  0.055523 avg.loss:  1.035399 ETA:   0h24m40s 44.5% words/sec/thread:  161395 lr:  0.055457 avg.loss:  1.034789 ETA:   0h24m38s 44.9% words/sec/thread:  161453 lr:  0.055134 avg.loss:  1.031992 ETA:   0h24m29s 45.1% words/sec/thread:  161471 lr:  0.054862 avg.loss:  1.029781 ETA:   0h24m21s 45.4% words/sec/thread:  161364 lr:  0.054631 avg.loss:  1.027887 ETA:   0h24m16savg.loss:  1.027026 ETA:   0h24m14s 45.6% words/sec/thread:  161396 lr:  0.054386 avg.loss:  1.025567 ETA:   0h24m 9savg.loss:  1.024916 ETA:   0h24m 7s 45.9% words/sec/t

Progress:  87.9% words/sec/thread:  154220 lr:  0.012088 avg.loss:  0.655178 ETA:   0h 5m37s 67.6% words/sec/thread:  158666 lr:  0.032439 avg.loss:  0.836139 ETA:   0h14m39s 67.6% words/sec/thread:  158576 lr:  0.032376 avg.loss:  0.835424 ETA:   0h14m38s 67.7% words/sec/thread:  158541 lr:  0.032330 avg.loss:  0.834916 ETA:   0h14m37s 67.8% words/sec/thread:  158533 lr:  0.032158 avg.loss:  0.832998 ETA:   0h14m32s14m31s 68.2% words/sec/thread:  158489 lr:  0.031821 avg.loss:  0.829244 ETA:   0h14m23sm21s 68.5% words/sec/thread:  158461 lr:  0.031535 avg.loss:  0.826102 ETA:   0h14m16s 68.6% words/sec/thread:  158430 lr:  0.031423 avg.loss:  0.824855 ETA:   0h14m13savg.loss:  0.824071 ETA:   0h14m11s 0.031140 avg.loss:  0.821765 ETA:   0h14m 5s 68.9% words/sec/thread:  158380 lr:  0.031061 avg.loss:  0.820906 ETA:   0h14m 3s 69.2% words/sec/thread:  158336 lr:  0.030829 avg.loss:  0.818402 ETA:   0h13m57sm52s 69.6% words/sec/thread:  158262 lr:  0.030387 avg.loss:  0.813639 ETA:   0h

Progress: 100.0% words/sec/thread:  152326 lr:  0.000022 avg.loss:  0.579445 ETA:   0h 0m 0ssec/thread:  154191 lr:  0.011866 avg.loss:  0.653606 ETA:   0h 5m31s 88.5% words/sec/thread:  154172 lr:  0.011514 avg.loss:  0.651131 ETA:   0h 5m21s 88.6% words/sec/thread:  154154 lr:  0.011398 avg.loss:  0.650320 ETA:   0h 5m18s 88.7% words/sec/thread:  154152 lr:  0.011328 avg.loss:  0.649835 ETA:   0h 5m16s 88.7% words/sec/thread:  154152 lr:  0.011268 avg.loss:  0.649427 ETA:   0h 5m14s 88.9% words/sec/thread:  154146 lr:  0.011148 avg.loss:  0.648598 ETA:   0h 5m11s 5s 0.010841 avg.loss:  0.646474 ETA:   0h 5m 2s 89.4% words/sec/thread:  154089 lr:  0.010595 avg.loss:  0.644776 ETA:   0h 4m55s 89.9% words/sec/thread:  154084 lr:  0.010120 avg.loss:  0.641573 ETA:   0h 4m42s 89.9% words/sec/thread:  154075 lr:  0.010055 avg.loss:  0.641140 ETA:   0h 4m40s 90.2% words/sec/thread:  154054 lr:  0.009840 avg.loss:  0.639694 ETA:   0h 4m34s 90.2% words/sec/thread:  154032 lr:  0.009783 avg.lo

Total training time: 0 hours 47 minutes 5 seconds


Progress: 100.0% words/sec/thread:  152315 lr:  0.000000 avg.loss:  0.579319 ETA:   0h 0m 0s


In [139]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_8.bin")


In [140]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_8.bin")


In [141]:
# relatedness model cbow 8
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.22747771372874676


In [142]:
# word analogy model cbow 8
word_analogy(model_cbow)

true_predicted 19 0.0018973437187936888
false_predicted 9995 0.9981026562812063
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [143]:
# Membuat model word embedding fasttext menggunakan skip gram 8

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.1, dim=30, ws=5, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:   8.9% words/sec/thread:   57867 lr:  0.091101 avg.loss:  0.853423 ETA:   1h52m53s

Progress:  16.4% words/sec/thread:   57211 lr:  0.083641 avg.loss:  0.580747 ETA:   1h44m50s  9.0% words/sec/thread:   57906 lr:  0.091033 avg.loss:  0.850302 ETA:   1h52m43s  9.0% words/sec/thread:   57894 lr:  0.090977 avg.loss:  0.847648 ETA:   1h52m41s  9.1% words/sec/thread:   57906 lr:  0.090950 avg.loss:  0.846430 ETA:   1h52m37s  1h52m33s  9.1% words/sec/thread:   57827 lr:  0.090863 avg.loss:  0.842442 ETA:   1h52m40s  9.2% words/sec/thread:   57787 lr:  0.090754 avg.loss:  0.837511 ETA:   1h52m37s  9.3% words/sec/thread:   57772 lr:  0.090725 avg.loss:  0.836223 ETA:   1h52m36s  9.3% words/sec/thread:   57788 lr:  0.090686 avg.loss:  0.834452 ETA:   1h52m31s  9.4% words/sec/thread:   57799 lr:  0.090648 avg.loss:  0.832804 ETA:   1h52m27s  9.5% words/sec/thread:   57758 lr:  0.090498 avg.loss:  0.826162 ETA:   1h52m21s  9.6% words/sec/thread:   57763 lr:  0.090406 avg.loss:  0.822293 ETA:   1h52m13savg.loss:  0.821161 ETA:   1h52m13s9.7% words/sec/thread:   57728 lr:  0.09025

Progress:  24.6% words/sec/thread:   57333 lr:  0.075428 avg.loss:  0.394785 ETA:   1h34m20s 16.4% words/sec/thread:   57209 lr:  0.083625 avg.loss:  0.580200 ETA:   1h44m49s 16.4% words/sec/thread:   57192 lr:  0.083605 avg.loss:  0.579508 ETA:   1h44m49s 16.5% words/sec/thread:   57200 lr:  0.083509 avg.loss:  0.576244 ETA:   1h44m41s 16.5% words/sec/thread:   57198 lr:  0.083482 avg.loss:  0.575345 ETA:   1h44m39s 16.6% words/sec/thread:   57190 lr:  0.083420 avg.loss:  0.573286 ETA:   1h44m35s44m32s 16.7% words/sec/thread:   57205 lr:  0.083320 avg.loss:  0.569999 ETA:   1h44m26s 16.8% words/sec/thread:   57257 lr:  0.083175 avg.loss:  0.565229 ETA:   1h44m10s 16.9% words/sec/thread:   57270 lr:  0.083056 avg.loss:  0.561424 ETA:   1h43m59s 17.1% words/sec/thread:   57298 lr:  0.082905 avg.loss:  0.556651 ETA:   1h43m45s17.1% words/sec/thread:   57271 lr:  0.082890 avg.loss:  0.556170 ETA:   1h43m47s 17.2% words/sec/thread:   57244 lr:  0.082799 avg.loss:  0.553356 ETA:   1h43m43ss

Progress:  32.0% words/sec/thread:   57306 lr:  0.068049 avg.loss:  0.308543 ETA:   1h25m 9s 24.6% words/sec/thread:   57335 lr:  0.075418 avg.loss:  0.394629 ETA:   1h34m19s 24.6% words/sec/thread:   57338 lr:  0.075396 avg.loss:  0.394303 ETA:   1h34m17s 24.7% words/sec/thread:   57322 lr:  0.075258 avg.loss:  0.392224 ETA:   1h34m 8s7s 24.9% words/sec/thread:   57316 lr:  0.075092 avg.loss:  0.389758 ETA:   1h33m56s 25.0% words/sec/thread:   57323 lr:  0.074956 avg.loss:  0.387758 ETA:   1h33m45s 25.2% words/sec/thread:   57323 lr:  0.074830 avg.loss:  0.385949 ETA:   1h33m36s 25.2% words/sec/thread:   57321 lr:  0.074798 avg.loss:  0.385484 ETA:   1h33m34s 25.2% words/sec/thread:   57313 lr:  0.074760 avg.loss:  0.384935 ETA:   1h33m32s 25.3% words/sec/thread:   57326 lr:  0.074684 avg.loss:  0.383855 ETA:   1h33m25s 25.4% words/sec/thread:   57332 lr:  0.074558 avg.loss:  0.382083 ETA:   1h33m15s 25.6% words/sec/thread:   57345 lr:  0.074420 avg.loss:  0.380165 ETA:   1h33m 3s 25.

...

Progress:  88.1% words/sec/thread:   57050 lr:  0.011919 avg.loss:  0.119062 ETA:   0h14m58s 80.2% words/sec/thread:   57057 lr:  0.019833 avg.loss:  0.129622 ETA:   0h24m55s 80.2% words/sec/thread:   57056 lr:  0.019817 avg.loss:  0.129596 ETA:   0h24m54s 80.3% words/sec/thread:   57056 lr:  0.019748 avg.loss:  0.129488 ETA:   0h24m49s 80.3% words/sec/thread:   57052 lr:  0.019680 avg.loss:  0.129387 ETA:   0h24m44s 80.4% words/sec/thread:   57053 lr:  0.019627 avg.loss:  0.129309 ETA:   0h24m40s 80.4% words/sec/thread:   57055 lr:  0.019587 avg.loss:  0.129252 ETA:   0h24m37s 80.5% words/sec/thread:   57048 lr:  0.019536 avg.loss:  0.129179 ETA:   0h24m33s 80.6% words/sec/thread:   57052 lr:  0.019432 avg.loss:  0.129031 ETA:   0h24m25s ETA:   0h24m23s 80.7% words/sec/thread:   57054 lr:  0.019292 avg.loss:  0.128832 ETA:   0h24m14s 80.7% words/sec/thread:   57052 lr:  0.019253 avg.loss:  0.128777 ETA:   0h24m11savg.loss:  0.128734 ETA:   0h24m 9s 80.8% words/sec/thread:   57055 lr: 

Progress:  95.8% words/sec/thread:   56968 lr:  0.004228 avg.loss:  0.110173 ETA:   0h 5m19s 88.1% words/sec/thread:   57042 lr:  0.011879 avg.loss:  0.119011 ETA:   0h14m56s 88.2% words/sec/thread:   57047 lr:  0.011792 avg.loss:  0.118903 ETA:   0h14m49s 88.3% words/sec/thread:   57046 lr:  0.011724 avg.loss:  0.118819 ETA:   0h14m44s 88.4% words/sec/thread:   57044 lr:  0.011646 avg.loss:  0.118723 ETA:   0h14m38ss 88.5% words/sec/thread:   57048 lr:  0.011524 avg.loss:  0.118574 ETA:   0h14m29s 88.5% words/sec/thread:   57048 lr:  0.011492 avg.loss:  0.118535 ETA:   0h14m26s lr:  0.011434 avg.loss:  0.118465 ETA:   0h14m22s 88.7% words/sec/thread:   57045 lr:  0.011348 avg.loss:  0.118361 ETA:   0h14m15s 0.011244 avg.loss:  0.118233 ETA:   0h14m 7srogress:  88.8% words/sec/thread:   57055 lr:  0.011213 avg.loss:  0.118195 ETA:   0h14m 5s 88.9% words/sec/thread:   57059 lr:  0.011102 avg.loss:  0.118059 ETA:   0h13m57s 88.9% words/sec/thread:   57054 lr:  0.011055 avg.loss:  0.11800

Progress: 100.0% words/sec/thread:   56912 lr:  0.000007 avg.loss:  0.105774 ETA:   0h 0m 0s ETA:   0h 5m18s 95.8% words/sec/thread:   56968 lr:  0.004171 avg.loss:  0.110112 ETA:   0h 5m15s 95.9% words/sec/thread:   56966 lr:  0.004143 avg.loss:  0.110082 ETA:   0h 5m12s 95.9% words/sec/thread:   56964 lr:  0.004086 avg.loss:  0.110020 ETA:   0h 5m 8s 96.0% words/sec/thread:   56964 lr:  0.004049 avg.loss:  0.109980 ETA:   0h 5m 5savg.loss:  0.109950 ETA:   0h 5m 3s 96.1% words/sec/thread:   56963 lr:  0.003907 avg.loss:  0.109825 ETA:   0h 4m55s 96.1% words/sec/thread:   56962 lr:  0.003888 avg.loss:  0.109805 ETA:   0h 4m53s46s 96.3% words/sec/thread:   56958 lr:  0.003709 avg.loss:  0.109613 ETA:   0h 4m40savg.loss:  0.109577 ETA:   0h 4m37s 96.4% words/sec/thread:   56965 lr:  0.003586 avg.loss:  0.109481 ETA:   0h 4m30s 96.5% words/sec/thread:   56969 lr:  0.003480 avg.loss:  0.109370 ETA:   0h 4m22s11s 96.7% words/sec/thread:   56969 lr:  0.003306 avg.loss:  0.109183 ETA:   0h 4

Total training time: 2 hours 6 minutes 1 seconds


Progress: 100.0% words/sec/thread:   56911 lr:  0.000000 avg.loss:  0.105767 ETA:   0h 0m 0s


In [144]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_8.bin")


In [145]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_8.bin")


In [146]:
# relatedness model sg 8
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.19126783531863223


In [147]:
# word analogy model sg 8
word_analogy(model_sg)

true_predicted 21 0.002097064110245656
false_predicted 9993 0.9979029358897543
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [148]:
# Membuat model word embedding fasttext menggunakan CBOW 9

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.1, dim=30, ws=7, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  15.0% words/sec/thread:  115354 lr:  0.084969 avg.loss:  1.749166 ETA:   0h52m49s

Progress:  26.8% words/sec/thread:   97851 lr:  0.073188 avg.loss:  1.314520 ETA:   0h53m38s 15.1% words/sec/thread:  115207 lr:  0.084906 avg.loss:  1.747747 ETA:   0h52m50savg.loss:  1.746801 ETA:   0h52m53s 15.3% words/sec/thread:  114757 lr:  0.084743 avg.loss:  1.744529 ETA:   0h52m57s 15.4% words/sec/thread:  114293 lr:  0.084552 avg.loss:  1.741357 ETA:   0h53m 2s 15.5% words/sec/thread:  114153 lr:  0.084496 avg.loss:  1.740469 ETA:   0h53m 4s 15.6% words/sec/thread:  113826 lr:  0.084389 avg.loss:  1.738773 ETA:   0h53m 9s 15.7% words/sec/thread:  113642 lr:  0.084321 avg.loss:  1.737797 ETA:   0h53m12savg.loss:  1.737258 ETA:   0h53m14s 15.9% words/sec/thread:  113032 lr:  0.084109 avg.loss:  1.734754 ETA:   0h53m21s 15.9% words/sec/thread:  112916 lr:  0.084079 avg.loss:  1.734360 ETA:   0h53m23s 16.0% words/sec/thread:  112606 lr:  0.083968 avg.loss:  1.732647 ETA:   0h53m28s 16.3% words/sec/thread:  111906 lr:  0.083741 avg.loss:  1.728750 ETA:   0h53m39s 16.3% words/sec/t

Progress:  36.5% words/sec/thread:   93077 lr:  0.063454 avg.loss:  1.113676 ETA:   0h48m53s 26.9% words/sec/thread:   97811 lr:  0.073141 avg.loss:  1.313200 ETA:   0h53m37s 26.9% words/sec/thread:   97800 lr:  0.073118 avg.loss:  1.312555 ETA:   0h53m36s 26.9% words/sec/thread:   97775 lr:  0.073085 avg.loss:  1.311604 ETA:   0h53m36s 27.0% words/sec/thread:   97738 lr:  0.072991 avg.loss:  1.309050 ETA:   0h53m33s 27.1% words/sec/thread:   97723 lr:  0.072946 avg.loss:  1.307805 ETA:   0h53m31s 27.1% words/sec/thread:   97703 lr:  0.072901 avg.loss:  1.306523 ETA:   0h53m30s 27.2% words/sec/thread:   97650 lr:  0.072795 avg.loss:  1.303453 ETA:   0h53m27s 27.3% words/sec/thread:   97565 lr:  0.072654 avg.loss:  1.299711 ETA:   0h53m23s 27.5% words/sec/thread:   97512 lr:  0.072546 avg.loss:  1.296946 ETA:   0h53m20s 27.6% words/sec/thread:   97406 lr:  0.072393 avg.loss:  1.293052 ETA:   0h53m17s 27.7% words/sec/thread:   97367 lr:  0.072345 avg.loss:  1.291848 ETA:   0h53m16s 27.8%

Progress:  47.4% words/sec/thread:   90119 lr:  0.052618 avg.loss:  0.999578 ETA:   0h41m52s 36.8% words/sec/thread:   93016 lr:  0.063249 avg.loss:  1.110912 ETA:   0h48m45s 36.8% words/sec/thread:   92994 lr:  0.063226 avg.loss:  1.110624 ETA:   0h48m45s 36.8% words/sec/thread:   92974 lr:  0.063180 avg.loss:  1.110012 ETA:   0h48m43s37.0% words/sec/thread:   92901 lr:  0.063021 avg.loss:  1.107963 ETA:   0h48m38s 37.0% words/sec/thread:   92896 lr:  0.063005 avg.loss:  1.107750 ETA:   0h48m38s 37.0% words/sec/thread:   92875 lr:  0.062951 avg.loss:  1.107073 ETA:   0h48m36s 37.2% words/sec/thread:   92791 lr:  0.062830 avg.loss:  1.105489 ETA:   0h48m33s 37.3% words/sec/thread:   92763 lr:  0.062725 avg.loss:  1.104038 ETA:   0h48m29s 37.4% words/sec/thread:   92691 lr:  0.062574 avg.loss:  1.102208 ETA:   0h48m24s avg.loss:  1.101380 ETA:   0h48m22s  92644 lr:  0.062385 avg.loss:  1.099910 ETA:   0h48m17s15s 37.7% words/sec/thread:   92618 lr:  0.062280 avg.loss:  1.098660 ETA:   0

...

Progress:  89.0% words/sec/thread:   86574 lr:  0.010999 avg.loss:  0.659863 ETA:   0h 9m 6s 80.0% words/sec/thread:   87058 lr:  0.020042 avg.loss:  0.727870 ETA:   0h16m30s 80.0% words/sec/thread:   87052 lr:  0.019993 avg.loss:  0.727466 ETA:   0h16m28s 80.2% words/sec/thread:   87022 lr:  0.019821 avg.loss:  0.726062 ETA:   0h16m19s 80.3% words/sec/thread:   87008 lr:  0.019741 avg.loss:  0.725406 ETA:   0h16m16s 80.3% words/sec/thread:   87008 lr:  0.019696 avg.loss:  0.725046 ETA:   0h16m13s 80.5% words/sec/thread:   87000 lr:  0.019483 avg.loss:  0.723346 ETA:   0h16m 3s 80.6% words/sec/thread:   87003 lr:  0.019404 avg.loss:  0.722702 ETA:   0h15m59s15m57s words/sec/thread:   86999 lr:  0.019321 avg.loss:  0.722039 ETA:   0h15m55s 80.7% words/sec/thread:   87000 lr:  0.019274 avg.loss:  0.721662 ETA:   0h15m53s  87001 lr:  0.019234 avg.loss:  0.721343 ETA:   0h15m51s 80.8% words/sec/thread:   87000 lr:  0.019190 avg.loss:  0.720987 ETA:   0h15m49s 80.9% words/sec/thread:   8700

Progress:  97.7% words/sec/thread:   86183 lr:  0.002279 avg.loss:  0.603817 ETA:   0h 1m53s 89.1% words/sec/thread:   86573 lr:  0.010937 avg.loss:  0.659434 ETA:   0h 9m 3s 89.1% words/sec/thread:   86570 lr:  0.010854 avg.loss:  0.658855 ETA:   0h 8m59savg.loss:  0.658565 ETA:   0h 8m57s   86568 lr:  0.010789 avg.loss:  0.658392 ETA:   0h 8m56s 89.3% words/sec/thread:   86572 lr:  0.010715 avg.loss:  0.657874 ETA:   0h 8m52s 89.4% words/sec/thread:   86572 lr:  0.010648 avg.loss:  0.657417 ETA:   0h 8m49s  86570 lr:  0.010604 avg.loss:  0.657114 ETA:   0h 8m47s  86574 lr:  0.010492 avg.loss:  0.656346 ETA:   0h 8m41s 89.6% words/sec/thread:   86569 lr:  0.010397 avg.loss:  0.655706 ETA:   0h 8m36s 89.7% words/sec/thread:   86565 lr:  0.010311 avg.loss:  0.655120 ETA:   0h 8m32s 89.7% words/sec/thread:   86565 lr:  0.010270 avg.loss:  0.654838 ETA:   0h 8m30s 89.8% words/sec/thread:   86562 lr:  0.010228 avg.loss:  0.654549 ETA:   0h 8m28s 89.8% words/sec/thread:   86558 lr:  0.01018

Progress: 100.0% words/sec/thread:   86074 lr:  0.000000 avg.loss:  0.590471 ETA:   0h 0m 0sh 1m50s 98.0% words/sec/thread:   86165 lr:  0.002007 avg.loss:  0.602204 ETA:   0h 1m40s 98.0% words/sec/thread:   86163 lr:  0.001968 avg.loss:  0.601968 ETA:   0h 1m38s 98.1% words/sec/thread:   86148 lr:  0.001882 avg.loss:  0.601455 ETA:   0h 1m33s 98.2% words/sec/thread:   86148 lr:  0.001840 avg.loss:  0.601204 ETA:   0h 1m31s0.600964 ETA:   0h 1m29s/sec/thread:   86145 lr:  0.001758 avg.loss:  0.600719 ETA:   0h 1m27s 98.4% words/sec/thread:   86136 lr:  0.001551 avg.loss:  0.599481 ETA:   0h 1m17s 98.6% words/sec/thread:   86142 lr:  0.001421 avg.loss:  0.598735 ETA:   0h 1m10s98.6% words/sec/thread:   86142 lr:  0.001380 avg.loss:  0.598500 ETA:   0h 1m 8s 98.7% words/sec/thread:   86144 lr:  0.001258 avg.loss:  0.597782 ETA:   0h 1m 2s 98.8% words/sec/thread:   86145 lr:  0.001174 avg.loss:  0.597281 ETA:   0h 0m58s 99.0% words/sec/thread:   86144 lr:  0.001047 avg.loss:  0.596533 ETA

Total training time: 1 hours 23 minutes 19 seconds


Progress: 100.0% words/sec/thread:   86073 lr:  0.000000 avg.loss:  0.590469 ETA:   0h 0m 0s


In [149]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_9.bin")


In [150]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_9.bin")


In [151]:
# relatedness model cbow 9
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.22681816871908803


In [152]:
# word analogy model cbow 9
word_analogy(model_cbow)

true_predicted 21 0.002097064110245656
false_predicted 9993 0.9979029358897543
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [153]:
# Membuat model word embedding fasttext menggunakan skip gram 9

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.1, dim=30, ws=7, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:   2.9% words/sec/thread:   29297 lr:  0.097120 avg.loss:  1.308364 ETA:   3h57m42s

Progress:   5.6% words/sec/thread:   29288 lr:  0.094414 avg.loss:  0.908226 ETA:   3h51m 9s  2.9% words/sec/thread:   29297 lr:  0.097115 avg.loss:  1.306747 ETA:   3h57m42s  2.9% words/sec/thread:   29294 lr:  0.097100 avg.loss:  1.302307 ETA:   3h57m41s  2.9% words/sec/thread:   29302 lr:  0.097084 avg.loss:  1.297997 ETA:   3h57m34s  2.9% words/sec/thread:   29302 lr:  0.097070 avg.loss:  1.293992 ETA:   3h57m33s  3.0% words/sec/thread:   29297 lr:  0.097017 avg.loss:  1.279833 ETA:   3h57m27s  0.096980 avg.loss:  1.270012 ETA:   3h57m18s  3.1% words/sec/thread:   29308 lr:  0.096937 avg.loss:  1.258782 ETA:   3h57m10s  3.1% words/sec/thread:   29300 lr:  0.096923 avg.loss:  1.255131 ETA:   3h57m12s  3.1% words/sec/thread:   29288 lr:  0.096871 avg.loss:  1.242102 ETA:   3h57m10s  3.2% words/sec/thread:   29289 lr:  0.096850 avg.loss:  1.237104 ETA:   3h57m 6ss  3.2% words/sec/thread:   29294 lr:  0.096804 avg.loss:  1.226132 ETA:   3h56m57s  3.2% words/sec/thread:   29308 lr:  0.0

Progress:   8.8% words/sec/thread:   29256 lr:  0.091188 avg.loss:  0.752539 ETA:   3h43m30s  29293 lr:  0.094399 avg.loss:  0.907036 ETA:   3h51m 4sh51m 2s  5.7% words/sec/thread:   29266 lr:  0.094340 avg.loss:  0.902689 ETA:   3h51m 9s  5.7% words/sec/thread:   29263 lr:  0.094320 avg.loss:  0.901272 ETA:   3h51m 7s  5.7% words/sec/thread:   29259 lr:  0.094312 avg.loss:  0.900671 ETA:   3h51m 8s  29255 lr:  0.094297 avg.loss:  0.899573 ETA:   3h51m 7s% words/sec/thread:   29261 lr:  0.094268 avg.loss:  0.897512 ETA:   3h51m 0s  5.8% words/sec/thread:   29266 lr:  0.094238 avg.loss:  0.895366 ETA:   3h50m54s  5.8% words/sec/thread:   29264 lr:  0.094224 avg.loss:  0.894387 ETA:   3h50m52s ETA:   3h50m44s  5.9% words/sec/thread:   29260 lr:  0.094148 avg.loss:  0.889174 ETA:   3h50m43s  5.9% words/sec/thread:   29267 lr:  0.094141 avg.loss:  0.888725 ETA:   3h50m39s  5.9% words/sec/thread:   29269 lr:  0.094121 avg.loss:  0.887336 ETA:   3h50m35s  5.9% words/sec/thread:   29274 lr:  

Progress:  12.1% words/sec/thread:   29285 lr:  0.087886 avg.loss:  0.595152 ETA:   3h35m11s  8.8% words/sec/thread:   29260 lr:  0.091173 avg.loss:  0.752101 ETA:   3h43m26s  8.8% words/sec/thread:   29254 lr:  0.091153 avg.loss:  0.751500 ETA:   3h43m25s  8.9% words/sec/thread:   29253 lr:  0.091123 avg.loss:  0.750597 ETA:   3h43m21savg.loss:  0.750142 ETA:   3h43m19s 8.9% words/sec/thread:   29255 lr:  0.091092 avg.loss:  0.749623 ETA:   3h43m16svg.loss:  0.749196 ETA:   3h43m15s  9.0% words/sec/thread:   29261 lr:  0.091034 avg.loss:  0.747938 ETA:   3h43m 5s  9.0% words/sec/thread:   29261 lr:  0.091013 avg.loss:  0.747357 ETA:   3h43m 2s  9.0% words/sec/thread:   29262 lr:  0.090991 avg.loss:  0.746738 ETA:   3h42m58ss  9.0% words/sec/thread:   29269 lr:  0.090962 avg.loss:  0.745858 ETA:   3h42m51s3h42m49s  9.1% words/sec/thread:   29267 lr:  0.090933 avg.loss:  0.745015 ETA:   3h42m47savg.loss:  0.744658 ETA:   3h42m49s% words/sec/thread:   29292 lr:  0.090800 avg.loss:  0.741

...

Progress:  56.3% words/sec/thread:   29619 lr:  0.043661 avg.loss:  0.147950 ETA:   1h45m42s 52.7% words/sec/thread:   29663 lr:  0.047318 avg.loss:  0.157187 ETA:   1h54m23s 52.7% words/sec/thread:   29663 lr:  0.047304 avg.loss:  0.157151 ETA:   1h54m21s 52.7% words/sec/thread:   29663 lr:  0.047289 avg.loss:  0.157114 ETA:   1h54m18s 52.7% words/sec/thread:   29663 lr:  0.047276 avg.loss:  0.157078 ETA:   1h54m17s 52.8% words/sec/thread:   29661 lr:  0.047237 avg.loss:  0.156977 ETA:   1h54m11s 52.8% words/sec/thread:   29661 lr:  0.047216 avg.loss:  0.156920 ETA:   1h54m 8s 52.8% words/sec/thread:   29662 lr:  0.047172 avg.loss:  0.156805 ETA:   1h54m 2s 52.8% words/sec/thread:   29661 lr:  0.047157 avg.loss:  0.156766 ETA:   1h54m 0s 52.9% words/sec/thread:   29660 lr:  0.047113 avg.loss:  0.156648 ETA:   1h53m54s 52.9% words/sec/thread:   29659 lr:  0.047091 avg.loss:  0.156590 ETA:   1h53m51s 53.0% words/sec/thread:   29639 lr:  0.046967 avg.loss:  0.156263 ETA:   1h53m37s53.0% 

Progress:  76.5% words/sec/thread:   34194 lr:  0.023521 avg.loss:  0.112713 ETA:   0h49m19s 56.4% words/sec/thread:   29620 lr:  0.043631 avg.loss:  0.147879 ETA:   1h45m37s 56.4% words/sec/thread:   29620 lr:  0.043627 avg.loss:  0.147869 ETA:   1h45m36s  1h45m31s 56.5% words/sec/thread:   29622 lr:  0.043494 avg.loss:  0.147557 ETA:   1h45m17s 56.5% words/sec/thread:   29622 lr:  0.043481 avg.loss:  0.147527 ETA:   1h45m15s  0.043457 avg.loss:  0.147472 ETA:   1h45m11s 56.6% words/sec/thread:   29623 lr:  0.043436 avg.loss:  0.147423 ETA:   1h45m 8s 56.6% words/sec/thread:   29624 lr:  0.043424 avg.loss:  0.147395 ETA:   1h45m 6s   29622 lr:  0.043405 avg.loss:  0.147352 ETA:   1h45m 4s 56.7% words/sec/thread:   29616 lr:  0.043333 avg.loss:  0.147184 ETA:   1h44m55s9604 lr:  0.043212 avg.loss:  0.146906 ETA:   1h44m40s:  0.146677 ETA:   1h44m27slr:  0.043081 avg.loss:  0.146600 ETA:   1h44m23s  29595 lr:  0.042982 avg.loss:  0.146374 ETA:   1h44m 8s 0.042954 avg.loss:  0.146311 ETA

Progress:  95.9% words/sec/thread:   37744 lr:  0.004133 avg.loss:  0.092331 ETA:   0h 7m51s lr:  0.023363 avg.loss:  0.112508 ETA:   0h48m56s  34274 lr:  0.023194 avg.loss:  0.112286 ETA:   0h48m31s0.022919 avg.loss:  0.111928 ETA:   0h47m51s% words/sec/thread:   34355 lr:  0.022860 avg.loss:  0.111852 ETA:   0h47m42s 77.5% words/sec/thread:   34434 lr:  0.022530 avg.loss:  0.111433 ETA:   0h46m55s 0.110531 ETA:   0h45m 9s 78.4% words/sec/thread:   34663 lr:  0.021587 avg.loss:  0.110278 ETA:   0h44m39s 78.9% words/sec/thread:   34786 lr:  0.021068 avg.loss:  0.109659 ETA:   0h43m25s 79.0% words/sec/thread:   34796 lr:  0.021028 avg.loss:  0.109610 ETA:   0h43m20s 79.0% words/sec/thread:   34812 lr:  0.020956 avg.loss:  0.109526 ETA:   0h43m10s 79.6% words/sec/thread:   34948 lr:  0.020378 avg.loss:  0.108841 ETA:   0h41m48s% words/sec/thread:   35209 lr:  0.019275 avg.loss:  0.107524 ETA:   0h39m15s 81.1% words/sec/thread:   35294 lr:  0.018905 avg.loss:  0.107081 ETA:   0h38m24s 81.

Progress: 100.0% words/sec/thread:   37571 lr:  0.000000 avg.loss:  0.088788 ETA:   0h 0m 0s 95.9% words/sec/thread:   37741 lr:  0.004074 avg.loss:  0.092277 ETA:   0h 7m44s 95.9% words/sec/thread:   37740 lr:  0.004069 avg.loss:  0.092273 ETA:   0h 7m43s 96.0% words/sec/thread:   37738 lr:  0.004044 avg.loss:  0.092250 ETA:   0h 7m41s 96.0% words/sec/thread:   37735 lr:  0.004002 avg.loss:  0.092213 ETA:   0h 7m36s 96.0% words/sec/thread:   37735 lr:  0.003989 avg.loss:  0.092202 ETA:   0h 7m34s 96.0% words/sec/thread:   37733 lr:  0.003978 avg.loss:  0.092192 ETA:   0h 7m33savg.loss:  0.092180 ETA:   0h 7m32s 96.1% words/sec/thread:   37724 lr:  0.003903 avg.loss:  0.092125 ETA:   0h 7m25s 96.1% words/sec/thread:   37722 lr:  0.003886 avg.loss:  0.092110 ETA:   0h 7m23s 96.1% words/sec/thread:   37718 lr:  0.003850 avg.loss:  0.092078 ETA:   0h 7m19s 96.2% words/sec/thread:   37713 lr:  0.003803 avg.loss:  0.092037 ETA:   0h 7m13s 96.2% words/sec/thread:   37708 lr:  0.003766 avg.lo

Total training time: 3 hours 10 minutes 54 seconds


In [154]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_9.bin")


In [155]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_9.bin")


In [156]:
# relatedness model sg 9
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.18086325634028183


In [157]:
# word analogy model sg 9
word_analogy(model_sg)

true_predicted 20 0.0019972039145196726
false_predicted 9994 0.9980027960854804
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [158]:
# Membuat model word embedding fasttext menggunakan CBOW 10

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.1, dim=30, ws=3, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  17.4% words/sec/thread:  126494 lr:  0.082620 avg.loss:  1.632449 ETA:   0h46m50s

Progress:  50.6% words/sec/thread:  131501 lr:  0.049378 avg.loss:  0.902725 ETA:   0h26m55s 17.5% words/sec/thread:  126527 lr:  0.082510 avg.loss:  1.625896 ETA:   0h46m45s 17.6% words/sec/thread:  126503 lr:  0.082441 avg.loss:  1.621922 ETA:   0h46m43s 18.6% words/sec/thread:  127107 lr:  0.081448 avg.loss:  1.566190 ETA:   0h45m56s 18.8% words/sec/thread:  127219 lr:  0.081239 avg.loss:  1.555494 ETA:   0h45m47s 18.9% words/sec/thread:  127197 lr:  0.081097 avg.loss:  1.548015 ETA:   0h45m43s 19.0% words/sec/thread:  127227 lr:  0.081005 avg.loss:  1.543084 ETA:   0h45m39s 19.0% words/sec/thread:  127230 lr:  0.080959 avg.loss:  1.540729 ETA:   0h45m37s45m34s 19.4% words/sec/thread:  127303 lr:  0.080602 avg.loss:  1.523204 ETA:   0h45m24s 19.5% words/sec/thread:  127312 lr:  0.080530 avg.loss:  1.519735 ETA:   0h45m21s 20.6% words/sec/thread:  127767 lr:  0.079359 avg.loss:  1.466463 ETA:   0h44m32s 20.7% words/sec/thread:  127753 lr:  0.079271 avg.loss:  1.462834 ETA:   0h44m29s

Progress:  71.6% words/sec/thread:  129836 lr:  0.028422 avg.loss:  0.759972 ETA:   0h15m41s 50.8% words/sec/thread:  131483 lr:  0.049233 avg.loss:  0.901540 ETA:   0h26m51s 50.8% words/sec/thread:  131484 lr:  0.049185 avg.loss:  0.901145 ETA:   0h26m49s 51.4% words/sec/thread:  131512 lr:  0.048589 avg.loss:  0.895884 ETA:   0h26m29s 51.7% words/sec/thread:  131511 lr:  0.048265 avg.loss:  0.893286 ETA:   0h26m19s 52.0% words/sec/thread:  131549 lr:  0.048006 avg.loss:  0.890967 ETA:   0h26m10s 52.3% words/sec/thread:  131593 lr:  0.047717 avg.loss:  0.888636 ETA:   0h26m 0s ETA:   0h25m54s 52.5% words/sec/thread:  131568 lr:  0.047459 avg.loss:  0.886579 ETA:   0h25m51s 52.7% words/sec/thread:  131566 lr:  0.047341 avg.loss:  0.885673 ETA:   0h25m48s 52.7% words/sec/thread:  131560 lr:  0.047272 avg.loss:  0.885134 ETA:   0h25m45s 52.9% words/sec/thread:  131571 lr:  0.047061 avg.loss:  0.883262 ETA:   0h25m38s 53.4% words/sec/thread:  131616 lr:  0.046636 avg.loss:  0.879907 ETA: 

Progress:  89.6% words/sec/thread:  128085 lr:  0.010375 avg.loss:  0.635895 ETA:   0h 5m48s 71.6% words/sec/thread:  129824 lr:  0.028373 avg.loss:  0.759727 ETA:   0h15m40s 71.7% words/sec/thread:  129819 lr:  0.028317 avg.loss:  0.759425 ETA:   0h15m38s 71.8% words/sec/thread:  129811 lr:  0.028241 avg.loss:  0.759019 ETA:   0h15m36s 71.9% words/sec/thread:  129815 lr:  0.028095 avg.loss:  0.758085 ETA:   0h15m31s avg.loss:  0.757318 ETA:   0h15m26s 72.1% words/sec/thread:  129816 lr:  0.027875 avg.loss:  0.756899 ETA:   0h15m23s 72.5% words/sec/thread:  129843 lr:  0.027454 avg.loss:  0.754636 ETA:   0h15m 9s 72.7% words/sec/thread:  129851 lr:  0.027314 avg.loss:  0.753901 ETA:   0h15m 5s 72.8% words/sec/thread:  129850 lr:  0.027247 avg.loss:  0.753539 ETA:   0h15m 2savg.loss:  0.753177 ETA:   0h15m 0s 73.0% words/sec/thread:  129841 lr:  0.027032 avg.loss:  0.752264 ETA:   0h14m55s 73.0% words/sec/thread:  129845 lr:  0.026959 avg.loss:  0.751878 ETA:   0h14m53s 73.2% words/sec/

Progress:  99.1% words/sec/thread:  124512 lr:  0.000887 avg.loss:  0.576891 ETA:   0h 0m30s 89.7% words/sec/thread:  128053 lr:  0.010298 avg.loss:  0.635361 ETA:   0h 5m45s 5m44s% words/sec/thread:  128029 lr:  0.010199 avg.loss:  0.634700 ETA:   0h 5m42s 89.8% words/sec/thread:  128027 lr:  0.010182 avg.loss:  0.634584 ETA:   0h 5m42s 89.9% words/sec/thread:  128017 lr:  0.010108 avg.loss:  0.634074 ETA:   0h 5m39s  0h 5m35s 90.2% words/sec/thread:  128016 lr:  0.009824 avg.loss:  0.632174 ETA:   0h 5m30s 128014 lr:  0.009747 avg.loss:  0.631661 ETA:   0h 5m27s 90.4% words/sec/thread:  128015 lr:  0.009609 avg.loss:  0.630727 ETA:   0h 5m22s 90.5% words/sec/thread:  128008 lr:  0.009538 avg.loss:  0.630251 ETA:   0h 5m20s 90.6% words/sec/thread:  127970 lr:  0.009406 avg.loss:  0.629342 ETA:   0h 5m16s 0.628778 ETA:   0h 5m13s 90.8% words/sec/thread:  127953 lr:  0.009194 avg.loss:  0.627939 ETA:   0h 5m 9s  0h 5m 4s 91.6% words/sec/thread:  127912 lr:  0.008398 avg.loss:  0.622688 

Total training time: 0 hours 57 minutes 41 seconds


Prog

In [159]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_10.bin")


Progress:  99.4% words/sec/thread:  124469 lr:  0.0

In [160]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_10.bin")


In [161]:
# relatedness model cbow 10
print(relatedness(model_cbow))

Progress:  99.7% words/sec/thread:  124430 lr:  0.000344 avg.loss:  0.573810 ETA:   0h 0m11s

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.21481862506301877


In [162]:
# word analogy model cbow 10
word_analogy(model_cbow)

Progress: 100.0% words/sec/thread:  124407 lr:  0.000000 avg.loss:  0.571907 ETA:   0h 0m 0s11 avg.loss:  0.573070 ETA:   0h 0m 7s:  99.8% words/sec/thread:  124423 lr:  0.000155 avg.loss:  0.572775 ETA:   0h 0m 5s


true_predicted 32 0.003195526263231476
false_predicted 9982 0.9968044737367685
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [163]:
# Membuat model word embedding fasttext menggunakan skip gram 10

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.1, dim=30, ws=3, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  15.5% words/sec/thread:   52891 lr:  0.084514 avg.loss:  0.779388 ETA:   1h54m34s

Progress:  28.3% words/sec/thread:   54251 lr:  0.071690 avg.loss:  0.503976 ETA:   1h34m45s 15.5% words/sec/thread:   52901 lr:  0.084500 avg.loss:  0.779127 ETA:   1h54m32s 15.5% words/sec/thread:   52909 lr:  0.084481 avg.loss:  0.778801 ETA:   1h54m29s 15.6% words/sec/thread:   52948 lr:  0.084414 avg.loss:  0.777616 ETA:   1h54m19s 15.7% words/sec/thread:   53013 lr:  0.084330 avg.loss:  0.776184 ETA:   1h54m 4s 16.6% words/sec/thread:   53789 lr:  0.083432 avg.loss:  0.760382 ETA:   1h51m13s 16.8% words/sec/thread:   53985 lr:  0.083166 avg.loss:  0.756270 ETA:   1h50m28s 16.9% words/sec/thread:   53989 lr:  0.083071 avg.loss:  0.754726 ETA:   1h50m20s 17.5% words/sec/thread:   54335 lr:  0.082546 avg.loss:  0.746349 ETA:   1h48m56s0.082310 avg.loss:  0.742949 ETA:   1h48m19s 17.8% words/sec/thread:   54588 lr:  0.082181 avg.loss:  0.741010 ETA:   1h47m57s 18.4% words/sec/thread:   55028 lr:  0.081632 avg.loss:  0.732522 ETA:   1h46m22s 18.8% words/sec/thread:   55328 lr:  0.0812

Progress:  39.0% words/sec/thread:   54711 lr:  0.061046 avg.loss:  0.372681 ETA:   1h20m 0s 28.3% words/sec/thread:   54249 lr:  0.071678 avg.loss:  0.503772 ETA:   1h34m44savg.loss:  0.503316 ETA:   1h34m42s   1h34m40s8.4% words/sec/thread:   54239 lr:  0.071573 avg.loss:  0.501942 ETA:   1h34m37s99120 ETA:   1h34m25s 28.7% words/sec/thread:   54229 lr:  0.071253 avg.loss:  0.496466 ETA:   1h34m13s 28.8% words/sec/thread:   54236 lr:  0.071211 avg.loss:  0.495751 ETA:   1h34m 9savg.loss:  0.495276 ETA:   1h34m 6s 28.9% words/sec/thread:   54244 lr:  0.071077 avg.loss:  0.493499 ETA:   1h33m57s 29.0% words/sec/thread:   54251 lr:  0.071031 avg.loss:  0.492743 ETA:   1h33m53s 29.0% words/sec/thread:   54258 lr:  0.070999 avg.loss:  0.492238 ETA:   1h33m50s 29.0% words/sec/thread:   54261 lr:  0.070975 avg.loss:  0.491832 ETA:   1h33m47s29.1% words/sec/thread:   54272 lr:  0.070921 avg.loss:  0.490978 ETA:   1h33m42svg.loss:  0.490517 ETA:   1h33m39s 29.4% words/sec/thread:   54257 lr: 

Progress:  51.1% words/sec/thread:   55529 lr:  0.048893 avg.loss:  0.288128 ETA:   1h 3m 8sogress:  39.2% words/sec/thread:   54635 lr:  0.060819 avg.loss:  0.370658 ETA:   1h19m49s.060794 avg.loss:  0.370433 ETA:   1h19m47s 39.3% words/sec/thread:   54644 lr:  0.060678 avg.loss:  0.369410 ETA:   1h19m37s 39.3% words/sec/thread:   54649 lr:  0.060653 avg.loss:  0.369200 ETA:   1h19m35s 39.4% words/sec/thread:   54663 lr:  0.060576 avg.loss:  0.368532 ETA:   1h19m27s 39.5% words/sec/thread:   54678 lr:  0.060501 avg.loss:  0.367878 ETA:   1h19m20s avg.loss:  0.367243 ETA:   1h19m13s 39.6% words/sec/thread:   54703 lr:  0.060384 avg.loss:  0.366856 ETA:   1h19m 9s 39.7% words/sec/thread:   54711 lr:  0.060346 avg.loss:  0.366522 ETA:   1h19m 5s 39.8% words/sec/thread:   54746 lr:  0.060201 avg.loss:  0.365265 ETA:   1h18m51savg.loss:  0.365061 ETA:   1h18m49s 39.9% words/sec/thread:   54734 lr:  0.060066 avg.loss:  0.364089 ETA:   1h18m41savg.loss:  0.363895 ETA:   1h18m40s 40.0% words/

...

Progress:  94.4% words/sec/thread:   54999 lr:  0.005569 avg.loss:  0.163005 ETA:   0h 7m15s 88.2% words/sec/thread:   55253 lr:  0.011774 avg.loss:  0.173832 ETA:   0h15m16s0.173802 ETA:   0h15m15s 88.3% words/sec/thread:   55249 lr:  0.011743 avg.loss:  0.173774 ETA:   0h15m14s 88.3% words/sec/thread:   55247 lr:  0.011728 avg.loss:  0.173746 ETA:   0h15m13s% words/sec/thread:   55239 lr:  0.011660 avg.loss:  0.173621 ETA:   0h15m 8s 88.4% words/sec/thread:   55236 lr:  0.011630 avg.loss:  0.173565 ETA:   0h15m 5s 88.4% words/sec/thread:   55232 lr:  0.011579 avg.loss:  0.173469 ETA:   0h15m 1s 88.5% words/sec/thread:   55231 lr:  0.011546 avg.loss:  0.173410 ETA:   0h14m59s avg.loss:  0.173317 ETA:   0h14m55s 88.5% words/sec/thread:   55229 lr:  0.011469 avg.loss:  0.173267 ETA:   0h14m53s 88.7% words/sec/thread:   55223 lr:  0.011272 avg.loss:  0.172903 ETA:   0h14m38s 88.8% words/sec/thread:   55215 lr:  0.011179 avg.loss:  0.172731 ETA:   0h14m31s 88.9% words/sec/thread:   55215 

Progress:  99.3% words/sec/thread:   54806 lr:  0.000730 avg.loss:  0.155440 ETA:   0h 0m57s 94.4% words/sec/thread:   54998 lr:  0.005557 avg.loss:  0.162985 ETA:   0h 7m14s 94.6% words/sec/thread:   54996 lr:  0.005414 avg.loss:  0.162752 ETA:   0h 7m 3s 94.6% words/sec/thread:   54994 lr:  0.005396 avg.loss:  0.162722 ETA:   0h 7m 2s 94.6% words/sec/thread:   54993 lr:  0.005371 avg.loss:  0.162682 ETA:   0h 7m 0s  0.005332 avg.loss:  0.162618 ETA:   0h 6m57s 94.7% words/sec/thread:   54988 lr:  0.005283 avg.loss:  0.162539 ETA:   0h 6m53s 94.7% words/sec/thread:   54987 lr:  0.005267 avg.loss:  0.162512 ETA:   0h 6m52s 94.8% words/sec/thread:   54985 lr:  0.005235 avg.loss:  0.162460 ETA:   0h 6m49s 94.8% words/sec/thread:   54984 lr:  0.005219 avg.loss:  0.162433 ETA:   0h 6m48s 94.8% words/sec/thread:   54982 lr:  0.005187 avg.loss:  0.162381 ETA:   0h 6m45s 94.8% words/sec/thread:   54981 lr:  0.005170 avg.loss:  0.162352 ETA:   0h 6m44s43s 94.8% words/sec/thread:   54981 lr:  0

Progress: 100.0% words/sec/thread:   54773 lr:  0.000016 avg.loss:  0.154384 ETA:   0h 0m 1s 99.3% words/sec/thread:   54805 lr:  0.000686 avg.loss:  0.155375 ETA:   0h 0m53s 99.3% words/sec/thread:   54804 lr:  0.000669 avg.loss:  0.155349 ETA:   0h 0m52s 99.4% words/sec/thread:   54802 lr:  0.000606 avg.loss:  0.155257 ETA:   0h 0m47s ETA:   0h 0m46s 99.4% words/sec/thread:   54801 lr:  0.000574 avg.loss:  0.155209 ETA:   0h 0m45s 99.4% words/sec/thread:   54799 lr:  0.000557 avg.loss:  0.155185 ETA:   0h 0m43s 99.5% words/sec/thread:   54798 lr:  0.000474 avg.loss:  0.155062 ETA:   0h 0m37s 99.6% words/sec/thread:   54793 lr:  0.000396 avg.loss:  0.154945 ETA:   0h 0m31s.154782 ETA:   0h 0m22s14s0.154584 ETA:   0h 0m11s 99.9% words/sec/thread:   54776 lr:  0.000119 avg.loss:  0.154535 ETA:   0h 0m 9s 99.9% words/sec/thread:   54775 lr:  0.000075 avg.loss:  0.154469 ETA:   0h 0m 5s100.0% words/sec/thread:   54774 lr:  0.000043 avg.loss:  0.154423 ETA:   0h 0m 3s

Total training time: 2 hours 10 minutes 58 seconds


Progress: 100.0% words/sec/thread:   54773 lr:  0.000000 avg.loss:  0.154358 ETA:   0h 0m 0s


In [164]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_10.bin")


In [165]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_10.bin")


In [166]:
# relatedness model sg 10
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.19955161351024894


In [167]:
# word analogy model sg 10
word_analogy(model_sg)

true_predicted 22 0.00219692430597164
false_predicted 9992 0.9978030756940284
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [168]:
# Membuat model word embedding fasttext menggunakan CBOW 11

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.1, dim=30, ws=9, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:   6.8% words/sec/thread:   71368 lr:  0.093202 avg.loss:  1.998430 ETA:   1h33m38s

Progress:  16.2% words/sec/thread:   75056 lr:  0.083799 avg.loss:  1.767314 ETA:   1h20m 3s  6.8% words/sec/thread:   71383 lr:  0.093178 avg.loss:  1.998326 ETA:   1h33m36s  6.8% words/sec/thread:   71368 lr:  0.093154 avg.loss:  1.998105 ETA:   1h33m35s  6.9% words/sec/thread:   71382 lr:  0.093124 avg.loss:  1.997406 ETA:   1h33m32s  6.9% words/sec/thread:   71424 lr:  0.093065 avg.loss:  1.996489 ETA:   1h33m26s  7.0% words/sec/thread:   71432 lr:  0.093039 avg.loss:  1.996111 ETA:   1h33m23s  7.0% words/sec/thread:   71403 lr:  0.093020 avg.loss:  1.995878 ETA:   1h33m25s  7.0% words/sec/thread:   71378 lr:  0.092982 avg.loss:  1.995616 ETA:   1h33m24s  7.1% words/sec/thread:   71457 lr:  0.092872 avg.loss:  1.992587 ETA:   1h33m11s  7.2% words/sec/thread:   71554 lr:  0.092759 avg.loss:  1.987972 ETA:   1h32m57s.985785 ETA:   1h32m51s  7.3% words/sec/thread:   71614 lr:  0.092676 avg.loss:  1.984636 ETA:   1h32m47s  7.6% words/sec/thread:   71862 lr:  0.092424 avg.loss:  1.97519

Progress:  25.6% words/sec/thread:   73865 lr:  0.074351 avg.loss:  1.364763 ETA:   1h12m10s 16.2% words/sec/thread:   75049 lr:  0.083783 avg.loss:  1.766189 ETA:   1h20m 3s 16.2% words/sec/thread:   75044 lr:  0.083755 avg.loss:  1.764307 ETA:   1h20m 1s75043 lr:  0.083724 avg.loss:  1.762185 ETA:   1h20m 0s 16.4% words/sec/thread:   75066 lr:  0.083622 avg.loss:  1.755428 ETA:   1h19m52s 16.5% words/sec/thread:   75093 lr:  0.083531 avg.loss:  1.749377 ETA:   1h19m45s 16.5% words/sec/thread:   75088 lr:  0.083497 avg.loss:  1.747188 ETA:   1h19m44s 16.5% words/sec/thread:   75096 lr:  0.083477 avg.loss:  1.745868 ETA:   1h19m42s words/sec/thread:   75102 lr:  0.083447 avg.loss:  1.743777 ETA:   1h19m40s 16.8% words/sec/thread:   75153 lr:  0.083232 avg.loss:  1.729559 ETA:   1h19m25s 16.8% words/sec/thread:   75146 lr:  0.083201 avg.loss:  1.727450 ETA:   1h19m23s 16.9% words/sec/thread:   75145 lr:  0.083143 avg.loss:  1.723521 ETA:   1h19m20s 16.9% words/sec/thread:   75139 lr:  0

Progress:  41.0% words/sec/thread:   80741 lr:  0.059032 avg.loss:  1.117522 ETA:   0h52m25s 25.9% words/sec/thread:   73713 lr:  0.074110 avg.loss:  1.358161 ETA:   1h12m 5s1h11m58s 26.2% words/sec/thread:   73548 lr:  0.073801 avg.loss:  1.350151 ETA:   1h11m57s 26.3% words/sec/thread:   73522 lr:  0.073734 avg.loss:  1.348451 ETA:   1h11m54s 26.3% words/sec/thread:   73508 lr:  0.073679 avg.loss:  1.347055 ETA:   1h11m52s   73486 lr:  0.073632 avg.loss:  1.345870 ETA:   1h11m51s6.4% words/sec/thread:   73450 lr:  0.073550 avg.loss:  1.343836 ETA:   1h11m48sords/sec/thread:   73266 lr:  0.073169 avg.loss:  1.334111 ETA:   1h11m36s  1.331437 ETA:   1h11m31s 27.3% words/sec/thread:   73170 lr:  0.072714 avg.loss:  1.322685 ETA:   1h11m15s 27.4% words/sec/thread:   73169 lr:  0.072629 avg.loss:  1.320634 ETA:   1h11m10s 27.4% words/sec/thread:   73170 lr:  0.072601 avg.loss:  1.319948 ETA:   1h11m 9s 27.5% words/sec/thread:   73175 lr:  0.072523 avg.loss:  1.318067 ETA:   1h11m 4s 27.5%

...

Progress:  70.6% words/sec/thread:   72541 lr:  0.029404 avg.loss:  0.819593 ETA:   0h29m 4s/thread:   73395 lr:  0.040267 avg.loss:  0.947164 ETA:   0h39m20sess:  59.9% words/sec/thread:   73357 lr:  0.040129 avg.loss:  0.945275 ETA:   0h39m13sss:  59.9% words/sec/thread:   73347 lr:  0.040063 avg.loss:  0.944375 ETA:   0h39m10sloss:  0.937352 ETA:   0h38m43sss:  0.935633 ETA:   0h38m36s lr:  0.039178 avg.loss:  0.932646 ETA:   0h38m23s 60.9% words/sec/thread:   73191 lr:  0.039096 avg.loss:  0.931548 ETA:   0h38m18s 61.0% words/sec/thread:   73193 lr:  0.039028 avg.loss:  0.930662 ETA:   0h38m14savg.loss:  0.930174 ETA:   0h38m12s61.2% words/sec/thread:   73190 lr:  0.038823 avg.loss:  0.927970 ETA:   0h38m 2s0h37m56s 61.6% words/sec/thread:   73058 lr:  0.038413 avg.loss:  0.922681 ETA:   0h37m42s 61.6% words/sec/thread:   73059 lr:  0.038381 avg.loss:  0.922265 ETA:   0h37m40s 61.6% words/sec/thread:   73060 lr:  0.038350 avg.loss:  0.921872 ETA:   0h37m38s 61.8% words/sec/thread: 

Progress:  78.0% words/sec/thread:   71879 lr:  0.021989 avg.loss:  0.750391 ETA:   0h21m56s 70.6% words/sec/thread:   72541 lr:  0.029384 avg.loss:  0.819379 ETA:   0h29m 2s 70.6% words/sec/thread:   72543 lr:  0.029356 avg.loss:  0.819102 ETA:   0h29m 1s 70.8% words/sec/thread:   72554 lr:  0.029178 avg.loss:  0.817291 ETA:   0h28m50s 70.9% words/sec/thread:   72548 lr:  0.029088 avg.loss:  0.816375 ETA:   0h28m45s 70.9% words/sec/thread:   72547 lr:  0.029057 avg.loss:  0.816064 ETA:   0h28m43s 71.0% words/sec/thread:   72548 lr:  0.029025 avg.loss:  0.815739 ETA:   0h28m41s  72547 lr:  0.028999 avg.loss:  0.815469 ETA:   0h28m39s 71.0% words/sec/thread:   72546 lr:  0.028968 avg.loss:  0.815152 ETA:   0h28m38s 71.1% words/sec/thread:   72546 lr:  0.028908 avg.loss:  0.814541 ETA:   0h28m34s 71.3% words/sec/thread:   72565 lr:  0.028731 avg.loss:  0.812728 ETA:   0h28m23s 71.5% words/sec/thread:   72571 lr:  0.028476 avg.loss:  0.810191 ETA:   0h28m 8sm 6s 71.7% words/sec/thread:   

Progress:  92.3% words/sec/thread:   74984 lr:  0.007729 avg.loss:  0.644016 ETA:   0h 7m23s 78.0% words/sec/thread:   71877 lr:  0.021974 avg.loss:  0.750262 ETA:   0h21m55s avg.loss:  0.749477 ETA:   0h21m49s 78.1% words/sec/thread:   71874 lr:  0.021852 avg.loss:  0.749227 ETA:   0h21m48s 78.3% words/sec/thread:   71851 lr:  0.021672 avg.loss:  0.747691 ETA:   0h21m37s  71834 lr:  0.021522 avg.loss:  0.746434 ETA:   0h21m29s 78.5% words/sec/thread:   71834 lr:  0.021489 avg.loss:  0.746162 ETA:   0h21m27s 0.745644 ETA:   0h21m23s 78.6% words/sec/thread:   71827 lr:  0.021397 avg.loss:  0.745389 ETA:   0h21m21s 78.6% words/sec/thread:   71826 lr:  0.021369 avg.loss:  0.745152 ETA:   0h21m20s 78.7% words/sec/thread:   71825 lr:  0.021336 avg.loss:  0.744873 ETA:   0h21m18s 78.7% words/sec/thread:   71824 lr:  0.021288 avg.loss:  0.744477 ETA:   0h21m15s 78.7% words/sec/thread:   71824 lr:  0.021275 avg.loss:  0.744367 ETA:   0h21m14savg.loss:  0.744094 ETA:   0h21m12s 79.0% words/sec/

Progress: 100.0% words/sec/thread:   77340 lr:  0.000000 avg.loss:  0.596518 ETA:   0h 0m 0s 92.4% words/sec/thread:   75012 lr:  0.007637 avg.loss:  0.643439 ETA:   0h 7m18s 92.4% words/sec/thread:   75026 lr:  0.007566 avg.loss:  0.642977 ETA:   0h 7m13s 92.6% words/sec/thread:   75081 lr:  0.007400 avg.loss:  0.641900 ETA:   0h 7m 4s lr:  0.007295 avg.loss:  0.641215 ETA:   0h 6m57s 92.9% words/sec/thread:   75177 lr:  0.007089 avg.loss:  0.639873 ETA:   0h 6m45s 6m43s 93.1% words/sec/thread:   75233 lr:  0.006908 avg.loss:  0.638708 ETA:   0h 6m35s 93.2% words/sec/thread:   75253 lr:  0.006843 avg.loss:  0.638293 ETA:   0h 6m31s 93.3% words/sec/thread:   75286 lr:  0.006730 avg.loss:  0.637561 ETA:   0h 6m24savg.loss:  0.637170 ETA:   0h 6m20s 93.9% words/sec/thread:   75474 lr:  0.006090 avg.loss:  0.633471 ETA:   0h 5m47s 94.4% words/sec/thread:   75641 lr:  0.005624 avg.loss:  0.630543 ETA:   0h 5m19s 94.7% words/sec/thread:   75767 lr:  0.005269 avg.loss:  0.628296 ETA:   0h 4m

Total training time: 1 hours 32 minutes 46 seconds


In [169]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_11.bin")


In [170]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_11.bin")


In [171]:
# relatedness model cbow 11
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.22793775249457895


In [172]:
# word analogy model cbow 11
word_analogy(model_cbow)

true_predicted 15 0.0014979029358897543
false_predicted 9999 0.9985020970641102
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [173]:
# Membuat model word embedding fasttext menggunakan skip gram 11

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.1, dim=30, ws=9, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  10.8% words/sec/thread:   49585 lr:  0.089220 avg.loss:  0.547542 ETA:   2h 9m 1s

Progress:  22.4% words/sec/thread:   47766 lr:  0.077612 avg.loss:  0.290415 ETA:   1h56m30s 10.8% words/sec/thread:   49594 lr:  0.089183 avg.loss:  0.545812 ETA:   2h 8m56s 11.3% words/sec/thread:   49876 lr:  0.088706 avg.loss:  0.525034 ETA:   2h 7m32s11.3% words/sec/thread:   49892 lr:  0.088653 avg.loss:  0.522816 ETA:   2h 7m25s 0.088638 avg.loss:  0.522168 ETA:   2h 7m22s 11.8% words/sec/thread:   50111 lr:  0.088247 avg.loss:  0.506547 ETA:   2h 6m16s 11.9% words/sec/thread:   50155 lr:  0.088135 avg.loss:  0.502351 ETA:   2h 6m 0s 12.2% words/sec/thread:   50269 lr:  0.087770 avg.loss:  0.488846 ETA:   2h 5m12s 12.3% words/sec/thread:   50252 lr:  0.087747 avg.loss:  0.488003 ETA:   2h 5m12s 12.3% words/sec/thread:   50240 lr:  0.087702 avg.loss:  0.486425 ETA:   2h 5m10sh 5m 1s 12.7% words/sec/thread:   50389 lr:  0.087346 avg.loss:  0.474197 ETA:   2h 4m18s 12.8% words/sec/thread:   50478 lr:  0.087162 avg.loss:  0.468182 ETA:   2h 3m49s 12.9% words/sec/thread:   50510 lr: 

Progress:  30.2% words/sec/thread:   40102 lr:  0.069794 avg.loss:  0.226089 ETA:   2h 4m48s 22.8% words/sec/thread:   47311 lr:  0.077249 avg.loss:  0.286537 ETA:   1h57m 5s 23.2% words/sec/thread:   46843 lr:  0.076845 avg.loss:  0.282282 ETA:   1h57m38s 23.2% words/sec/thread:   46791 lr:  0.076816 avg.loss:  0.281979 ETA:   1h57m43s 23.2% words/sec/thread:   46772 lr:  0.076804 avg.loss:  0.281855 ETA:   1h57m45s lr:  0.076662 avg.loss:  0.280399 ETA:   1h57m51s 23.5% words/sec/thread:   46506 lr:  0.076489 avg.loss:  0.278666 ETA:   1h57m56s 23.6% words/sec/thread:   46416 lr:  0.076410 avg.loss:  0.277885 ETA:   1h58m 2s 23.6% words/sec/thread:   46355 lr:  0.076362 avg.loss:  0.277413 ETA:   1h58m 7s 23.9% words/sec/thread:   46052 lr:  0.076069 avg.loss:  0.274597 ETA:   1h58m26s 24.0% words/sec/thread:   45973 lr:  0.075992 avg.loss:  0.273841 ETA:   1h58m31s 24.1% words/sec/thread:   45900 lr:  0.075924 avg.loss:  0.273169 ETA:   1h58m36s 24.3% words/sec/thread:   45972 lr:  

Progress:  33.9% words/sec/thread:   37712 lr:  0.066059 avg.loss:  0.205509 ETA:   2h 5m36s 30.2% words/sec/thread:   40082 lr:  0.069772 avg.loss:  0.225959 ETA:   2h 4m49s 30.2% words/sec/thread:   40060 lr:  0.069751 avg.loss:  0.225825 ETA:   2h 4m51s 30.3% words/sec/thread:   40050 lr:  0.069741 avg.loss:  0.225766 ETA:   2h 4m52s 30.3% words/sec/thread:   40008 lr:  0.069696 avg.loss:  0.225489 ETA:   2h 4m55s  2h 4m56s 30.3% words/sec/thread:   39966 lr:  0.069655 avg.loss:  0.225231 ETA:   2h 4m58s 30.4% words/sec/thread:   39938 lr:  0.069623 avg.loss:  0.225030 ETA:   2h 5m 0s 30.4% words/sec/thread:   39908 lr:  0.069589 avg.loss:  0.224823 ETA:   2h 5m 2s 30.4% words/sec/thread:   39899 lr:  0.069580 avg.loss:  0.224766 ETA:   2h 5m 3s 30.4% words/sec/thread:   39890 lr:  0.069571 avg.loss:  0.224713 ETA:   2h 5m 3s 30.4% words/sec/thread:   39871 lr:  0.069554 avg.loss:  0.224608 ETA:   2h 5m 5s avg.loss:  0.224123 ETA:   2h 5m10s0.069452 avg.loss:  0.223989 ETA:   2h 5m1

...

Progress:  87.7% words/sec/thread:   37222 lr:  0.012282 avg.loss:  0.091979 ETA:   0h23m39s 80.4% words/sec/thread:   38940 lr:  0.019587 avg.loss:  0.099416 ETA:   0h36m 4s 80.4% words/sec/thread:   38941 lr:  0.019559 avg.loss:  0.099385 ETA:   0h36m 0savg.loss:  0.099357 ETA:   0h35m58s56s avg.loss:  0.099276 ETA:   0h35m49s 80.7% words/sec/thread:   38969 lr:  0.019277 avg.loss:  0.099073 ETA:   0h35m28s0h35m25s 80.8% words/sec/thread:   38978 lr:  0.019197 avg.loss:  0.098986 ETA:   0h35m18s 80.9% words/sec/thread:   38984 lr:  0.019143 avg.loss:  0.098928 ETA:   0h35m12s 81.0% words/sec/thread:   38998 lr:  0.019038 avg.loss:  0.098817 ETA:   0h35m 0s 81.0% words/sec/thread:   39002 lr:  0.018999 avg.loss:  0.098775 ETA:   0h34m55s 81.1% words/sec/thread:   39009 lr:  0.018916 avg.loss:  0.098687 ETA:   0h34m46s ETA:   0h34m21s 81.4% words/sec/thread:   39049 lr:  0.018618 avg.loss:  0.098364 ETA:   0h34m11savg.loss:  0.098339 ETA:   0h34m 8s 81.5% words/sec/thread:   39063 lr: 

Progress:  93.6% words/sec/thread:   35300 lr:  0.006357 avg.loss:  0.086655 ETA:   0h12m54s 87.7% words/sec/thread:   37221 lr:  0.012274 avg.loss:  0.091972 ETA:   0h23m38s 87.7% words/sec/thread:   37219 lr:  0.012264 avg.loss:  0.091962 ETA:   0h23m37s 87.8% words/sec/thread:   37215 lr:  0.012244 avg.loss:  0.091944 ETA:   0h23m35s 87.8% words/sec/thread:   37209 lr:  0.012239 avg.loss:  0.091940 ETA:   0h23m35s 87.8% words/sec/thread:   37206 lr:  0.012237 avg.loss:  0.091937 ETA:   0h23m35savg.loss:  0.091930 ETA:   0h23m34s 87.9% words/sec/thread:   37111 lr:  0.012074 avg.loss:  0.091785 ETA:   0h23m19s 88.1% words/sec/thread:   37045 lr:  0.011936 avg.loss:  0.091656 ETA:   0h23m 6s 88.1% words/sec/thread:   37016 lr:  0.011886 avg.loss:  0.091610 ETA:   0h23m 1savg.loss:  0.091602 ETA:   0h23m 0s   37009 lr:  0.011874 avg.loss:  0.091598 ETA:   0h23m 0s 36992 lr:  0.011840 avg.loss:  0.091565 ETA:   0h22m57s 88.3% words/sec/thread:   36967 lr:  0.011741 avg.loss:  0.091471 E

Progress:  98.1% words/sec/thread:   35278 lr:  0.001908 avg.loss:  0.083030 ETA:   0h 3m52s 93.7% words/sec/thread:   35297 lr:  0.006332 avg.loss:  0.086635 ETA:   0h12m51s 93.7% words/sec/thread:   35294 lr:  0.006318 avg.loss:  0.086622 ETA:   0h12m50s   0h12m46s  0h12m45s 93.8% words/sec/thread:   35274 lr:  0.006222 avg.loss:  0.086541 ETA:   0h12m38s  35269 lr:  0.006181 avg.loss:  0.086507 ETA:   0h12m34s 93.8% words/sec/thread:   35267 lr:  0.006171 avg.loss:  0.086498 ETA:   0h12m32s93.8% words/sec/thread:   35266 lr:  0.006162 avg.loss:  0.086491 ETA:   0h12m31s 93.8% words/sec/thread:   35265 lr:  0.006151 avg.loss:  0.086481 ETA:   0h12m30s 93.9% words/sec/thread:   35262 lr:  0.006128 avg.loss:  0.086462 ETA:   0h12m27s 93.9% words/sec/thread:   35259 lr:  0.006104 avg.loss:  0.086441 ETA:   0h12m24s 93.9% words/sec/thread:   35257 lr:  0.006093 avg.loss:  0.086432 ETA:   0h12m23s 93.9% words/sec/thread:   35255 lr:  0.006074 avg.loss:  0.086416 ETA:   0h12m21sh12m20s 93.

Progress: 100.0% words/sec/thread:   35316 lr:  0.000000 avg.loss:  0.081568 ETA:   0h 0m 0s 98.1% words/sec/thread:   35279 lr:  0.001896 avg.loss:  0.083021 ETA:   0h 3m51s0h 3m37s  35277 lr:  0.001769 avg.loss:  0.082921 ETA:   0h 3m35s 98.3% words/sec/thread:   35275 lr:  0.001736 avg.loss:  0.082895 ETA:   0h 3m31s 98.3% words/sec/thread:   35272 lr:  0.001687 avg.loss:  0.082856 ETA:   0h 3m25s 98.4% words/sec/thread:   35276 lr:  0.001635 avg.loss:  0.082816 ETA:   0h 3m19s15sds/sec/thread:   35283 lr:  0.001565 avg.loss:  0.082760 ETA:   0h 3m10s 98.5% words/sec/thread:   35298 lr:  0.001451 avg.loss:  0.082672 ETA:   0h 2m56s 0.001365 avg.loss:  0.082607 ETA:   0h 2m46s 98.7% words/sec/thread:   35320 lr:  0.001251 avg.loss:  0.082519 ETA:   0h 2m32s 98.8% words/sec/thread:   35320 lr:  0.001249 avg.loss:  0.082518 ETA:   0h 2m32s 98.9% words/sec/thread:   35341 lr:  0.001096 avg.loss:  0.082400 ETA:   0h 2m13s 98.9% words/sec/thread:   35342 lr:  0.001089 avg.loss:  0.082395 

Total training time: 3 hours 23 minutes 4 seconds


In [174]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_11.bin")


In [175]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_11.bin")


In [176]:
# relatedness model sg 11
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.1734640219062538


In [177]:
# word analogy model sg 11
word_analogy(model_sg)

true_predicted 23 0.002296784501697623
false_predicted 9991 0.9977032154983024
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [178]:
# Membuat model word embedding fasttext menggunakan CBOW 12

# waktu mulai
start_time = time.time()

model_cbow = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='cbow', lr=0.1, dim=30, ws=11, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:  11.8% words/sec/thread:   57932 lr:  0.088209 avg.loss:  1.921764 ETA:   1h49m11s

Progress:  20.5% words/sec/thread:   61219 lr:  0.079523 avg.loss:  1.602626 ETA:   1h33m 8s 11.9% words/sec/thread:   58002 lr:  0.088126 avg.loss:  1.920321 ETA:   1h48m56s 11.9% words/sec/thread:   58022 lr:  0.088097 avg.loss:  1.919671 ETA:   1h48m52s 12.1% words/sec/thread:   58140 lr:  0.087947 avg.loss:  1.916630 ETA:   1h48m28s 12.2% words/sec/thread:   58205 lr:  0.087810 avg.loss:  1.914712 ETA:   1h48m10s 12.2% words/sec/thread:   58224 lr:  0.087780 avg.loss:  1.914194 ETA:   1h48m 6s 12.4% words/sec/thread:   58346 lr:  0.087641 avg.loss:  1.912126 ETA:   1h47m42s 12.4% words/sec/thread:   58412 lr:  0.087552 avg.loss:  1.910810 ETA:   1h47m28s 12.5% words/sec/thread:   58441 lr:  0.087518 avg.loss:  1.910457 ETA:   1h47m23s 12.7% words/sec/thread:   58584 lr:  0.087350 avg.loss:  1.908689 ETA:   1h46m55s 12.7% words/sec/thread:   58647 lr:  0.087267 avg.loss:  1.907736 ETA:   1h46m42s 12.8% words/sec/thread:   58703 lr:  0.087178 avg.loss:  1.906708 ETA:   1h46m29s 12.8%

Progress:  30.2% words/sec/thread:   61179 lr:  0.069834 avg.loss:  1.342581 ETA:   1h21m51s 20.6% words/sec/thread:   61274 lr:  0.079390 avg.loss:  1.597669 ETA:   1h32m54s 20.8% words/sec/thread:   61363 lr:  0.079177 avg.loss:  1.589894 ETA:   1h32m31s 21.0% words/sec/thread:   61422 lr:  0.079034 avg.loss:  1.584109 ETA:   1h32m16s 21.3% words/sec/thread:   61578 lr:  0.078675 avg.loss:  1.571329 ETA:   1h31m37s 21.4% words/sec/thread:   61623 lr:  0.078566 avg.loss:  1.567511 ETA:   1h31m25s 21.8% words/sec/thread:   61801 lr:  0.078182 avg.loss:  1.554657 ETA:   1h30m42s  0.078122 avg.loss:  1.552328 ETA:   1h30m39s 22.0% words/sec/thread:   61776 lr:  0.077968 avg.loss:  1.546890 ETA:   1h30m30s 22.1% words/sec/thread:   61788 lr:  0.077937 avg.loss:  1.545854 ETA:   1h30m26s 22.1% words/sec/thread:   61793 lr:  0.077912 avg.loss:  1.545038 ETA:   1h30m24s 22.1% words/sec/thread:   61733 lr:  0.077871 avg.loss:  1.543693 ETA:   1h30m27s 22.1% words/sec/thread:   61719 lr:  0.07

Progress:  43.0% words/sec/thread:   63255 lr:  0.057003 avg.loss:  1.146878 ETA:   1h 4m37s 30.2% words/sec/thread:   61177 lr:  0.069802 avg.loss:  1.341961 ETA:   1h21m49s 30.2% words/sec/thread:   61173 lr:  0.069783 avg.loss:  1.341610 ETA:   1h21m48s 30.3% words/sec/thread:   61142 lr:  0.069686 avg.loss:  1.339719 ETA:   1h21m43s 30.3% words/sec/thread:   61141 lr:  0.069653 avg.loss:  1.339064 ETA:   1h21m41s 30.6% words/sec/thread:   61271 lr:  0.069422 avg.loss:  1.334739 ETA:   1h21m14s 30.7% words/sec/thread:   61314 lr:  0.069344 avg.loss:  1.333336 ETA:   1h21m 5s 30.7% words/sec/thread:   61353 lr:  0.069293 avg.loss:  1.332420 ETA:   1h20m59s 30.7% words/sec/thread:   61365 lr:  0.069265 avg.loss:  1.331920 ETA:   1h20m56s 30.9% words/sec/thread:   61457 lr:  0.069089 avg.loss:  1.328276 ETA:   1h20m36s 31.0% words/sec/thread:   61515 lr:  0.069026 avg.loss:  1.327143 ETA:   1h20m27s 31.0% words/sec/thread:   61540 lr:  0.068985 avg.loss:  1.326367 ETA:   1h20m22s  6159

...

Progress:  70.8% words/sec/thread:   56060 lr:  0.029211 avg.loss:  0.828844 ETA:   0h37m21s 60.0% words/sec/thread:   58077 lr:  0.040029 avg.loss:  0.954033 ETA:   0h49m25s 60.0% words/sec/thread:   58071 lr:  0.039982 avg.loss:  0.953401 ETA:   0h49m22s 60.2% words/sec/thread:   58030 lr:  0.039810 avg.loss:  0.951148 ETA:   0h49m11s 60.2% words/sec/thread:   58029 lr:  0.039782 avg.loss:  0.950779 ETA:   0h49m 9s 60.2% words/sec/thread:   58026 lr:  0.039757 avg.loss:  0.950459 ETA:   0h49m 7s 60.3% words/sec/thread:   58021 lr:  0.039734 avg.loss:  0.950144 ETA:   0h49m 6s 60.3% words/sec/thread:   58014 lr:  0.039677 avg.loss:  0.949388 ETA:   0h49m 2s 0.039033 avg.loss:  0.940942 ETA:   0h48m23s words/sec/thread:   57799 lr:  0.038850 avg.loss:  0.938574 ETA:   0h48m11sss:  0.938246 ETA:   0h48m10s 61.6% words/sec/thread:   57716 lr:  0.038450 avg.loss:  0.933385 ETA:   0h47m46s 61.6% words/sec/thread:   57694 lr:  0.038366 avg.loss:  0.932314 ETA:   0h47m41s 61.7% words/sec/thr

Progress:  81.2% words/sec/thread:   57293 lr:  0.018806 avg.loss:  0.735497 ETA:   0h23m32s0h37m12s 70.9% words/sec/thread:   56046 lr:  0.029057 avg.loss:  0.827291 ETA:   0h37m10s 71.0% words/sec/thread:   56038 lr:  0.029007 avg.loss:  0.826793 ETA:   0h37m 7s 71.0% words/sec/thread:   56037 lr:  0.029007 avg.loss:  0.826791 ETA:   0h37m 7s 71.1% words/sec/thread:   56032 lr:  0.028940 avg.loss:  0.826134 ETA:   0h37m 2s 71.1% words/sec/thread:   56030 lr:  0.028924 avg.loss:  0.825967 ETA:   0h37m 1s71.1% words/sec/thread:   56025 lr:  0.028872 avg.loss:  0.825446 ETA:   0h36m57s 71.2% words/sec/thread:   56025 lr:  0.028819 avg.loss:  0.824919 ETA:   0h36m53s 71.2% words/sec/thread:   56021 lr:  0.028794 avg.loss:  0.824667 ETA:   0h36m51s.4% words/sec/thread:   55999 lr:  0.028648 avg.loss:  0.823196 ETA:   0h36m41s.loss:  0.822949 ETA:   0h36m39s 71.5% words/sec/thread:   55973 lr:  0.028523 avg.loss:  0.821960 ETA:   0h36m32s 71.7% words/sec/thread:   55934 lr:  0.028310 avg.l

Progress:  91.7% words/sec/thread:   59153 lr:  0.008316 avg.loss:  0.659567 ETA:   0h10m 4s 81.2% words/sec/thread:   57294 lr:  0.018789 avg.loss:  0.735365 ETA:   0h23m30savg.loss:  0.735083 ETA:   0h23m28s 81.3% words/sec/thread:   57314 lr:  0.018691 avg.loss:  0.734581 ETA:   0h23m23s 81.5% words/sec/thread:   57350 lr:  0.018467 avg.loss:  0.732843 ETA:   0h23m 5s 81.6% words/sec/thread:   57358 lr:  0.018414 avg.loss:  0.732437 ETA:   0h23m 1s 81.6% words/sec/thread:   57360 lr:  0.018392 avg.loss:  0.732267 ETA:   0h22m59s 81.7% words/sec/thread:   57371 lr:  0.018335 avg.loss:  0.731814 ETA:   0h22m55s 81.7% words/sec/thread:   57387 lr:  0.018277 avg.loss:  0.731354 ETA:   0h22m50s% words/sec/thread:   57453 lr:  0.018024 avg.loss:  0.729378 ETA:   0h22m29s22m27s 82.1% words/sec/thread:   57472 lr:  0.017895 avg.loss:  0.728358 ETA:   0h22m19s 82.2% words/sec/thread:   57496 lr:  0.017758 avg.loss:  0.727275 ETA:   0h22m 8s 82.4% words/sec/thread:   57521 lr:  0.017619 avg.l

Progress: 100.0% words/sec/thread:   60397 lr:  0.000000 avg.loss:  0.607501 ETA:   0h 0m 0s 91.7% words/sec/thread:   59164 lr:  0.008261 avg.loss:  0.659200 ETA:   0h10m 0s  59169 lr:  0.008231 avg.loss:  0.659004 ETA:   0h 9m58s91.8% words/sec/thread:   59172 lr:  0.008213 avg.loss:  0.658881 ETA:   0h 9m57s 0.008057 avg.loss:  0.657842 ETA:   0h 9m45s 0.007984 avg.loss:  0.657357 ETA:   0h 9m40s 92.0% words/sec/thread:   59215 lr:  0.007975 avg.loss:  0.657298 ETA:   0h 9m39s 92.2% words/sec/thread:   59234 lr:  0.007836 avg.loss:  0.656365 ETA:   0h 9m29s 92.3% words/sec/thread:   59257 lr:  0.007695 avg.loss:  0.655427 ETA:   0h 9m18s 92.4% words/sec/thread:   59267 lr:  0.007646 avg.loss:  0.655106 ETA:   0h 9m15s92.5% words/sec/thread:   59288 lr:  0.007484 avg.loss:  0.654065 ETA:   0h 9m 3s ETA:   0h 9m 3sad:   59304 lr:  0.007365 avg.loss:  0.653295 ETA:   0h 8m54s 92.7% words/sec/thread:   59316 lr:  0.007304 avg.loss:  0.652893 ETA:   0h 8m49s 92.8% words/sec/thread:   593

Total training time: 1 hours 58 minutes 46 seconds


In [179]:
# save model
model_cbow.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_12.bin")


In [180]:
# load model
model_cbow = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_cbow_fasttext_12.bin")


In [181]:
# relatedness model cbow 12
print(relatedness(model_cbow))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.2168835678093779


In [182]:
# word analogy model cbow 12
word_analogy(model_cbow)

true_predicted 14 0.0013980427401637707
false_predicted 10000 0.9986019572598362
banyak kata ada di model 3233
banyak kata tidak ada di model 0


In [183]:
# Membuat model word embedding fasttext menggunakan skip gram 12

# waktu mulai
start_time = time.time()

model_sg = fasttext.train_unsupervised(input='/Users/williamnehemia/Documents/Skripsi/TugasAkhir/WordEmbeddingModel/Data/dataCurr.txt', model='skipgram', lr=0.1, dim=30, ws=11, epoch=1000, minCount=1, minn=3, maxn=3, thread=10)

# waktu selesai
end_time = time.time()

# hitung waktu training
training_time = end_time - start_time

# convert ke jam, menit, detik
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)

print("Total training time:", hours, "hours", minutes, "minutes", seconds, "seconds")


Read 4M words
Number of words:  73023
Number of labels: 0


Progress:   3.5% words/sec/thread:   27311 lr:  0.096546 avg.loss:  0.920646 ETA:   4h13m29s

Progress:   7.4% words/sec/thread:   27316 lr:  0.092644 avg.loss:  0.657752 ETA:   4h 3m12s  3.5% words/sec/thread:   27308 lr:  0.096534 avg.loss:  0.919149 ETA:   4h13m29s  3.5% words/sec/thread:   27302 lr:  0.096525 avg.loss:  0.918121 ETA:   4h13m31s   27297 lr:  0.096505 avg.loss:  0.915700 ETA:   4h13m30s   4h13m33s  3.5% words/sec/thread:   27312 lr:  0.096473 avg.loss:  0.911917 ETA:   4h13m17s  3.6% words/sec/thread:   27297 lr:  0.096443 avg.loss:  0.908432 ETA:   4h13m20s  3.6% words/sec/thread:   27303 lr:  0.096395 avg.loss:  0.903073 ETA:   4h13m 9s0.096366 avg.loss:  0.899954 ETA:   4h12m54s  3.6% words/sec/thread:   27336 lr:  0.096353 avg.loss:  0.898639 ETA:   4h12m44s% words/sec/thread:   27353 lr:  0.096338 avg.loss:  0.896976 ETA:   4h12m33s  3.7% words/sec/thread:   27319 lr:  0.096317 avg.loss:  0.894888 ETA:   4h12m48s  3.7% words/sec/thread:   27352 lr:  0.096286 avg.loss:  0.891632 ETA:   4h12m25s  3.7% words/sec/thread:   27364 lr:  0.096274 avg.loss:  0.89

Progress:  11.7% words/sec/thread:   27362 lr:  0.088322 avg.loss:  0.436065 ETA:   3h51m27s  7.4% words/sec/thread:   27306 lr:  0.092608 avg.loss:  0.654837 ETA:   4h 3m11savg.loss:  0.653910 ETA:   4h 3m14s 7.4% words/sec/thread:   27299 lr:  0.092581 avg.loss:  0.652689 ETA:   4h 3m11s   4h 3m 8ss  7.5% words/sec/thread:   27311 lr:  0.092520 avg.loss:  0.647763 ETA:   4h 2m55s 2m50s  0.092458 avg.loss:  0.642948 ETA:   4h 2m40s  7.6% words/sec/thread:   27323 lr:  0.092426 avg.loss:  0.640474 ETA:   4h 2m34s% words/sec/thread:   27316 lr:  0.092398 avg.loss:  0.638379 ETA:   4h 2m33s  7.6% words/sec/thread:   27308 lr:  0.092380 avg.loss:  0.636998 ETA:   4h 2m34s  7.7% words/sec/thread:   27322 lr:  0.092343 avg.loss:  0.634223 ETA:   4h 2m21savg.loss:  0.633445 ETA:   4h 2m26s  27330 lr:  0.092274 avg.loss:  0.629088 ETA:   4h 2m 6s  7.7% words/sec/thread:   27320 lr:  0.092253 avg.loss:  0.627530 ETA:   4h 2m 8s  7.8% words/sec/thread:   27326 lr:  0.092231 avg.loss:  0.625918 

Progress:  15.4% words/sec/thread:   27407 lr:  0.084611 avg.loss:  0.345479 ETA:   3h41m22s 11.7% words/sec/thread:   27359 lr:  0.088290 avg.loss:  0.435056 ETA:   3h51m24s 11.7% words/sec/thread:   27364 lr:  0.088271 avg.loss:  0.434453 ETA:   3h51m19s  3h51m16s 11.8% words/sec/thread:   27364 lr:  0.088189 avg.loss:  0.431859 ETA:   3h51m 6s 11.8% words/sec/thread:   27352 lr:  0.088168 avg.loss:  0.431214 ETA:   3h51m 8s 11.9% words/sec/thread:   27357 lr:  0.088150 avg.loss:  0.430653 ETA:   3h51m 3s 11.9% words/sec/thread:   27360 lr:  0.088085 avg.loss:  0.428683 ETA:   3h50m51s 11.9% words/sec/thread:   27352 lr:  0.088067 avg.loss:  0.428110 ETA:   3h50m52savg.loss:  0.427746 ETA:   3h50m53s 12.0% words/sec/thread:   27357 lr:  0.088021 avg.loss:  0.426712 ETA:   3h50m43s 12.0% words/sec/thread:   27356 lr:  0.088014 avg.loss:  0.426497 ETA:   3h50m42s 12.0% words/sec/thread:   27359 lr:  0.087951 avg.loss:  0.424592 ETA:   3h50m30s 12.1% words/sec/thread:   27352 lr:  0.087

...

Progress:  90.5% words/sec/thread:   27160 lr:  0.009480 avg.loss:  0.078708 ETA:   0h25m 1s 86.4% words/sec/thread:   27158 lr:  0.013578 avg.loss:  0.081887 ETA:   0h35m51savg.loss:  0.081880 ETA:   0h35m49s 86.5% words/sec/thread:   27157 lr:  0.013532 avg.loss:  0.081851 ETA:   0h35m43s 86.5% words/sec/thread:   27158 lr:  0.013487 avg.loss:  0.081813 ETA:   0h35m36s 86.5% words/sec/thread:   27158 lr:  0.013476 avg.loss:  0.081804 ETA:   0h35m34sh35m33s 86.6% words/sec/thread:   27155 lr:  0.013440 avg.loss:  0.081775 ETA:   0h35m29s86.6% words/sec/thread:   27155 lr:  0.013417 avg.loss:  0.081755 ETA:   0h35m25s 86.6% words/sec/thread:   27155 lr:  0.013402 avg.loss:  0.081741 ETA:   0h35m23savg.loss:  0.081730 ETA:   0h35m21s86.6% words/sec/thread:   27155 lr:  0.013374 avg.loss:  0.081717 ETA:   0h35m18s 86.7% words/sec/thread:   27156 lr:  0.013348 avg.loss:  0.081695 ETA:   0h35m14s 86.7% words/sec/thread:   27155 lr:  0.013306 avg.loss:  0.081660 ETA:   0h35m 8s 86.7% words/

Progress:  94.3% words/sec/thread:   27186 lr:  0.005719 avg.loss:  0.075860 ETA:   0h15m 5s  27160 lr:  0.009473 avg.loss:  0.078702 ETA:   0h25m 0s lr:  0.009407 avg.loss:  0.078652 ETA:   0h24m50s27158 lr:  0.009347 avg.loss:  0.078605 ETA:   0h24m40s words/sec/thread:   27158 lr:  0.009308 avg.loss:  0.078573 ETA:   0h24m34s 90.7% words/sec/thread:   27159 lr:  0.009282 avg.loss:  0.078554 ETA:   0h24m30s 90.7% words/sec/thread:   27159 lr:  0.009261 avg.loss:  0.078539 ETA:   0h24m27s 90.8% words/sec/thread:   27160 lr:  0.009226 avg.loss:  0.078512 ETA:   0h24m21s 90.8% words/sec/thread:   27160 lr:  0.009215 avg.loss:  0.078503 ETA:   0h24m19s 90.9% words/sec/thread:   27160 lr:  0.009132 avg.loss:  0.078440 ETA:   0h24m 6s 0.009109 avg.loss:  0.078424 ETA:   0h24m 2s 90.9% words/sec/thread:   27161 lr:  0.009092 avg.loss:  0.078411 ETA:   0h24m 0s   27161 lr:  0.009071 avg.loss:  0.078394 ETA:   0h23m56s90.9% words/sec/thread:   27161 lr:  0.009063 avg.loss:  0.078388 ETA:   0h

Progress:  97.6% words/sec/thread:   27183 lr:  0.002379 avg.loss:  0.073606 ETA:   0h 6m16s94.3% words/sec/thread:   27186 lr:  0.005652 avg.loss:  0.075813 ETA:   0h14m54s words/sec/thread:   27185 lr:  0.005637 avg.loss:  0.075802 ETA:   0h14m52s 94.4% words/sec/thread:   27186 lr:  0.005596 avg.loss:  0.075773 ETA:   0h14m45s 94.5% words/sec/thread:   27185 lr:  0.005549 avg.loss:  0.075740 ETA:   0h14m38s 94.5% words/sec/thread:   27185 lr:  0.005532 avg.loss:  0.075729 ETA:   0h14m35s 94.5% words/sec/thread:   27184 lr:  0.005512 avg.loss:  0.075715 ETA:   0h14m32s% words/sec/thread:   27183 lr:  0.005470 avg.loss:  0.075685 ETA:   0h14m25s 94.5% words/sec/thread:   27183 lr:  0.005470 avg.loss:  0.075685 ETA:   0h14m25s 94.6% words/sec/thread:   27182 lr:  0.005442 avg.loss:  0.075665 ETA:   0h14m21s 94.6% words/sec/thread:   27183 lr:  0.005426 avg.loss:  0.075655 ETA:   0h14m18s 94.6% words/sec/thread:   27183 lr:  0.005414 avg.loss:  0.075647 ETA:   0h14m16s 94.6% words/sec/t

Progress: 100.0% words/sec/thread:   27187 lr:  0.000000 avg.loss:  0.072044 ETA:   0h 0m 0s 97.7% words/sec/thread:   27183 lr:  0.002327 avg.loss:  0.073571 ETA:   0h 6m 8s 97.7% words/sec/thread:   27183 lr:  0.002302 avg.loss:  0.073554 ETA:   0h 6m 4s 97.7% words/sec/thread:   27183 lr:  0.002294 avg.loss:  0.073548 ETA:   0h 6m 3s 97.8% words/sec/thread:   27181 lr:  0.002228 avg.loss:  0.073505 ETA:   0h 5m52s 97.8% words/sec/thread:   27182 lr:  0.002196 avg.loss:  0.073485 ETA:   0h 5m47s 97.8% words/sec/thread:   27181 lr:  0.002168 avg.loss:  0.073468 ETA:   0h 5m43s 97.8% words/sec/thread:   27180 lr:  0.002158 avg.loss:  0.073461 ETA:   0h 5m41s 97.9% words/sec/thread:   27180 lr:  0.002141 avg.loss:  0.073451 ETA:   0h 5m38s 97.9% words/sec/thread:   27180 lr:  0.002132 avg.loss:  0.073445 ETA:   0h 5m37s   27178 lr:  0.002118 avg.loss:  0.073435 ETA:   0h 5m35s 97.9% words/sec/thread:   27179 lr:  0.002080 avg.loss:  0.073411 ETA:   0h 5m29s 97.9% words/sec/thread:   271

Total training time: 4 hours 23 minutes 47 seconds


In [184]:
# save model
model_sg.save_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_12.bin")


In [185]:
# load model
model_sg = fasttext.load_model("/Users/williamnehemia/Documents/Skripsi/Fastext_model/SetelahPembagianData/model_sg_fasttext_12.bin")


In [186]:
# relatedness model sg 12
print(relatedness(model_sg))

banyak kata ada di model 2951
banyak kata tidak ada di model 0
0.16958730846416595


In [187]:
# word analogy model sg 12
word_analogy(model_sg)

true_predicted 22 0.00219692430597164
false_predicted 9992 0.9978030756940284
banyak kata ada di model 3233
banyak kata tidak ada di model 0
